# OSIC Pulmonary Fibrosis Progression - Plan

Goal: Win a medal (modified-laplace-log-likelihood as metric). Ship a strong baseline fast, lock CV, iterate with FE and robust modeling, then blend if time permits.

Plan:
- Environment checks: verify GPU availability (nvidia-smi), Python/package versions.
- Data audit: load train/test, inspect schema, target distribution, missingness, ID/time columns.
- Validation design:
  - Mirror test conditions; likely GroupKFold by patient (if patient column exists) or temporal split by Week if available.
  - Single deterministic folds saved to disk; evaluate with custom metric.
- Baseline models:
  - Fast baseline: Light tabular model (XGBoost/CatBoost with GPU if beneficial).
  - Features: numeric and categorical encoders; simple aggregates per patient/week if applicable.
- Metric implementation: implement modified-laplace-log-likelihood (per competition), validate vs trivial baselines.
- Iteration:
  - FE v1: core tabular features + patient aggregates; OOF tracking and logging.
  - FE v2: interactions, time deltas, trend features (per patient slopes).
  - Model tuning: limited HPO with early stopping; multiple seeds.
  - Ensembling: blend diverse models if CV shows complementary errors.
- Deliverables: submission.csv with correct format; verify shape and header.

Questions for Experts (next step):
- What CV protocol best mirrors leaderboard for this dataset? GroupKFold by patient? Number of folds?
- High-signal features others used (e.g., baseline FVC, week deltas, age/sex/smoking interaction)?
- Typical winning model choices and target transform tips?

Next: Run env check and quick EDA.

In [1]:
import os, sys, time, json, textwrap, subprocess as sp
import pandas as pd
import numpy as np

def sh(cmd):
    print(f"$ {cmd}", flush=True)
    return sp.run(cmd, shell=True, check=False)

t0 = time.time()
print("Env check: nvidia-smi")
sh("nvidia-smi || true")
print("Python:", sys.version)

train_path = 'train.csv'
test_path = 'test.csv'
ss_path = 'sample_submission.csv'

print("\nLoading CSVs ...", flush=True)
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
ss = pd.read_csv(ss_path)

print("\nTrain shape:", train.shape)
print("Train columns:", list(train.columns))
print(train.head(3))

print("\nTest shape:", test.shape)
print("Test columns:", list(test.columns))
print(test.head(3))

print("\nSample submission shape:", ss.shape)
print("Sample submission columns:", list(ss.columns))
print(ss.head(3))

# Try to infer key columns
cols = set(train.columns)
candidate_id_cols = [c for c in ['Patient','patient','id'] if c in cols]
candidate_week_cols = [c for c in ['Weeks','Week','weeks'] if c in cols]
target_cols = [c for c in ['FVC','target'] if c in cols]
demo_cols = [c for c in ['Age','Sex','SmokingStatus','Percent'] if c in cols]
print("\nDetected:")
print("  ID cols:", candidate_id_cols)
print("  Week cols:", candidate_week_cols)
print("  Target cols:", target_cols)
print("  Demographic cols:", [c for c in demo_cols if c in cols])

if target_cols:
    tcol = target_cols[0]
    print(f"\nTarget `{tcol}` describe:")
    print(train[tcol].describe())

# Patient stats
pid_col = None
for c in ['Patient','patient','id']:
    if c in train.columns:
        pid_col = c; break
if pid_col:
    n_pat = train[pid_col].nunique()
    print(f"\nPatients in train: {n_pat}")
    cnts = train[pid_col].value_counts()
    print("Per-patient visit count (5-number summary):", cnts.describe()[['min','25%','50%','75%','max']].to_dict())

# Submission format inference
need_sigma = False
if {'FVC','Confidence'}.issubset(set(ss.columns)):
    need_sigma = True
elif len(ss.columns) == 2 and {'Patient_Week'}.issubset(set(ss.columns)):
    # Often original format: Patient_Week, FVC
    need_sigma = 'Confidence' in ss.columns
print("\nSubmission seems to require sigma (Confidence):", need_sigma)

print(f"\nDone. Elapsed: {time.time()-t0:.2f}s", flush=True)

Env check: nvidia-smi
$ nvidia-smi || true


Wed Sep 24 05:29:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


Train shape: (1394, 7)
Train columns: ['Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus']
                     Patient  Weeks   FVC    Percent  Age   Sex SmokingStatus
0  ID00133637202223847701934     -2  3195  92.856312   83  Male  Never smoked
1  ID00133637202223847701934      2  3203  93.088817   83  Male  Never smoked
2  ID00133637202223847701934      4  3097  90.008138   83  Male  Never smoked

Test shape: (18, 7)
Test columns: ['Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus']
                     Patient  Weeks   FVC    Percent  Age     Sex  \
0  ID00014637202177757139317      0  3807  90.076661   56    Male   
1  ID00019637202178323708467     13  2100  92.858722   83  Female   
2  ID00047637202184938901501      2  3313  89.929425   68    Male   

  SmokingStatus  
0     Ex-smoker  
1     Ex-smoker  
2     Ex-smoker  

Sample submission shape: (1908, 3)
Sample submission columns: ['Patient_Week', 'FVC', 'Confidence']
                   Patie

In [2]:
import numpy as np, pandas as pd, os, pickle, time
from sklearn.model_selection import GroupKFold

# Reuse loaded train/test from previous cell if present; else load
if 'train' not in globals():
    train = pd.read_csv('train.csv')
if 'test' not in globals():
    test = pd.read_csv('test.csv')

PID = 'Patient'
WEEK = 'Weeks'
TARGET = 'FVC'

# Build and persist 5-fold GroupKFold by Patient
def make_group_folds(df, group_col=PID, n_splits=5, seed=42):
    gkf = GroupKFold(n_splits=n_splits)
    groups = df[group_col].values
    folds = []
    for fold, (tr_idx, va_idx) in enumerate(gkf.split(df, groups=groups)):
        folds.append((tr_idx, va_idx))
    return folds

folds_path = 'folds_groupk_patient.pkl'
if not os.path.exists(folds_path):
    folds = make_group_folds(train, PID, n_splits=5)
    with open(folds_path, 'wb') as f:
        pickle.dump(folds, f)
    print(f"Saved folds to {folds_path}")
else:
    with open(folds_path, 'rb') as f:
        folds = pickle.load(f)
    print(f"Loaded folds from {folds_path}")
print('Fold sizes (val counts):', [len(v) for _, v in folds])

# Modified Laplace Log Likelihood (per OSIC)
def laplace_log_likelihood(y_true, mu, sigma, sigma_floor=70.0, error_clip=1000.0):
    y_true = np.asarray(y_true, dtype=float)
    mu = np.asarray(mu, dtype=float)
    sigma = np.asarray(sigma, dtype=float)
    if sigma_floor is not None:
        sigma = np.maximum(sigma, float(sigma_floor))
    d = np.abs(y_true - mu)
    if error_clip is not None:
        d = np.minimum(d, float(error_clip))
    # row score: -log(2*sigma) - sqrt(2)*d/sigma
    row = -np.log(2.0 * sigma) - np.sqrt(2.0) * d / sigma
    return float(np.mean(row))

# Sanity check: trivial baseline
t0 = time.time()
global_mu = train[TARGET].median()
global_sigma = 70.0

oof_pred = np.full(len(train), global_mu, dtype=float)
oof_sigma = np.full(len(train), global_sigma, dtype=float)

score = laplace_log_likelihood(train[TARGET].values, oof_pred, oof_sigma)
print(f"Trivial constant baseline OOF (5xGroupK, constant across folds) Laplace LL: {score:.5f}")
print(f"Done metric+folds setup in {time.time()-t0:.2f}s")

Saved folds to folds_groupk_patient.pkl
Fold sizes (val counts): [282, 281, 275, 275, 281]
Trivial constant baseline OOF (5xGroupK, constant across folds) Laplace LL: -16.00958
Done metric+folds setup in 0.00s


In [9]:
import time, gc, sys, subprocess as sp
import numpy as np, pandas as pd, pickle, os
from sklearn.model_selection import GroupKFold

# Ensure catboost is installed
try:
    import catboost as cb
except Exception as e:
    print('Installing catboost...', flush=True)
    sp.run([sys.executable, '-m', 'pip', 'install', 'catboost==1.2.5'], check=True)
    import catboost as cb

PID, WEEK, TARGET = 'Patient','Weeks','FVC'
CAT_COLS = ['Sex','SmokingStatus']
NUM_COLS = ['Percent','Age']

if 'train' not in globals():
    train = pd.read_csv('train.csv')
if 'test' not in globals():
    test = pd.read_csv('test.csv')
with open('folds_groupk_patient.pkl','rb') as f:
    folds = pickle.load(f)

# Build features per-subset (leak-safe and avoids NaNs in val) with quick wins
def build_features_subset(df):
    # baseline and deltas
    basew = df.groupby(PID)[WEEK].transform('min')
    weeks_since = df[WEEK].values - basew.values
    abs_wsb = np.abs(weeks_since)
    # visit indices within subset
    visit_index = df.groupby(PID)[WEEK].rank(method='first', ascending=True).astype(int).values
    visits_count = df.groupby(PID)[WEEK].transform('count').values
    visits_remaining = (visits_count - visit_index).astype(int)
    # per-patient mean Percent within subset
    mean_percent = df.groupby(PID)['Percent'].transform('mean').values
    # sex numeric for interaction
    sex_num = (df['Sex'].values == 'Male').astype(int)
    feats = pd.DataFrame({
        'baseline_week': basew.values,
        'weeks_since_baseline': weeks_since,
        'wsb_sq': weeks_since**2,
        'abs_wsb': abs_wsb,
        'log_abs_wsb': np.log1p(abs_wsb),
        'visit_index': visit_index,
        'visits_remaining': visits_remaining,
        'mean_percent': mean_percent,
        'wsb_x_percent': weeks_since * df['Percent'].values,
        'age_x_sex': df['Age'].values * sex_num,
        'percent_x_age': df['Percent'].values * df['Age'].values,
        'sex_num': sex_num,
    })
    return feats

def cat_features_indices(df, cat_cols):
    return [df.columns.get_loc(c) for c in cat_cols if c in df.columns]

def last_k_mask(df_fold, pid_col=PID, week_col=WEEK, k=2):
    ord_idx = df_fold[[pid_col, week_col]].reset_index().sort_values([pid_col, week_col])
    ord_idx['rnk'] = ord_idx.groupby(pid_col)[week_col].rank(method='first', ascending=False)
    take_idx = set(ord_idx.loc[ord_idx['rnk'] <= k, 'index'].tolist())
    mask = df_fold.index.to_series().isin(take_idx).values
    return mask

def train_mu_sigma_oof(train, folds, params_mu=None, params_sigma=None, sigma_floor=70.0,
                       scale_grid=(0.8,1.0,1.2,1.4,1.6), a_grid=(0.0,10.0,30.0,50.0,80.0), b_grid=(0.3,0.5,1.0,1.5,2.0),
                       random_seed=42, temporal_k=2):
    n = len(train)
    oof_mu = np.zeros(n, dtype=float)
    oof_sigma_raw = np.zeros(n, dtype=float)
    oof_abs_wsb = np.zeros(n, dtype=float)
    models_mu, models_sigma = [], []
    t_start = time.time()
    for fi, (tr_idx, va_idx) in enumerate(folds):
        t0 = time.time()
        tr, va = train.iloc[tr_idx].copy(), train.iloc[va_idx].copy()
        # Per-subset features
        feats_train = build_features_subset(tr).reset_index(drop=True)
        feats_valid = build_features_subset(va).reset_index(drop=True)
        # Combine with raw cols
        X_tr = pd.concat([feats_train, tr[NUM_COLS + CAT_COLS].reset_index(drop=True)], axis=1)
        X_va = pd.concat([feats_valid, va[NUM_COLS + CAT_COLS].reset_index(drop=True)], axis=1)
        y_tr, y_va = tr[TARGET].values, va[TARGET].values
        # CatBoost mu
        task_type = 'GPU'
        try:
            cb.CatBoostRegressor(task_type=task_type)
        except Exception:
            task_type = 'CPU'
        if params_mu is None:
            params_mu = dict(depth=6, learning_rate=0.06, iterations=4000,
                             l2_leaf_reg=8.0, loss_function='Quantile:alpha=0.5',
                             bootstrap_type='Bernoulli', subsample=0.8,
                             random_seed=random_seed, early_stopping_rounds=200, task_type=task_type, verbose=200, allow_writing_files=False)
        cat_idx = cat_features_indices(X_tr, CAT_COLS)
        model_mu = cb.CatBoostRegressor(**params_mu)
        model_mu.fit(X_tr, y_tr, eval_set=(X_va, y_va), cat_features=cat_idx, verbose=200)
        mu_va = model_mu.predict(X_va)
        oof_mu[va_idx] = mu_va
        models_mu.append(model_mu)
        # Store abs_wsb for sigma floor calibration
        oof_abs_wsb[va_idx] = X_va['abs_wsb'].values
        # Sigma model: target = |residual|
        res_tr = np.abs(y_tr - model_mu.predict(X_tr))
        y_tr_sigma = res_tr
        if params_sigma is None:
            params_sigma = dict(depth=5, learning_rate=0.05, iterations=1200,
                                 l2_leaf_reg=6.0, loss_function='RMSE',
                                 bootstrap_type='Bernoulli', subsample=0.9,
                                 random_seed=random_seed, early_stopping_rounds=100, task_type=task_type, verbose=200, allow_writing_files=False)
        model_sigma = cb.CatBoostRegressor(**params_sigma)
        model_sigma.fit(X_tr, y_tr_sigma, eval_set=(X_va, np.abs(y_va - mu_va)), cat_features=cat_idx, verbose=200)
        sig_raw_va = model_sigma.predict(X_va)
        sig_raw_va = np.maximum(sig_raw_va, 5.0)  # tiny clamp for stability
        oof_sigma_raw[va_idx] = sig_raw_va
        models_sigma.append(model_sigma)
        elapsed = time.time()-t0
        print(f"Fold {fi} done in {elapsed:.1f}s", flush=True)
    # Clean arrays for robust post-process
    oof_abs_wsb_clean = np.nan_to_num(oof_abs_wsb, nan=0.0, posinf=0.0, neginf=0.0)
    oof_sigma_raw_clean = np.where(np.isfinite(oof_sigma_raw), oof_sigma_raw, np.nan)
    if np.isnan(oof_sigma_raw_clean).any():
        med = np.nanmedian(oof_sigma_raw_clean)
        if not np.isfinite(med):
            med = 100.0
        oof_sigma_raw_clean = np.nan_to_num(oof_sigma_raw_clean, nan=med, posinf=med, neginf=med)
    # Grid search for sigma post-process on last k val visits per patient within each fold
    best_score = -1e9
    best_params = (1.0, 0.0, 1.0)
    for s in scale_grid:
        for a in a_grid:
            for b in b_grid:
                total, count = 0.0, 0
                for tr_idx, va_idx in folds:
                    va_df = train.iloc[va_idx]
                    mask_k = last_k_mask(va_df, k=temporal_k)
                    if not mask_k.any():
                        mask_k = np.ones(len(va_idx), dtype=bool)
                    use_idx = np.array(va_idx)[mask_k]
                    if use_idx.size == 0:
                        continue
                    sigma = np.maximum(oof_sigma_raw_clean[use_idx] * s, a + b * oof_abs_wsb_clean[use_idx])
                    sigma = np.maximum(sigma, sigma_floor)
                    scr = laplace_log_likelihood(train[TARGET].values[use_idx],
                                                 oof_mu[use_idx], sigma,
                                                 sigma_floor=sigma_floor, error_clip=1000.0)
                    total += scr * use_idx.size
                    count += use_idx.size
                if count > 0:
                    avg_scr = total / count
                    if avg_scr > best_score:
                        best_score = avg_scr
                        best_params = (s, a, b)
    print(f"Best temporal OOF Laplace: {best_score:.5f} @ s={best_params[0]}, a={best_params[1]}, b={best_params[2]}")
    print(f"Total training time: {time.time()-t_start:.1f}s")
    return oof_mu, oof_sigma_raw_clean, oof_abs_wsb_clean, models_mu, models_sigma, best_params, best_score

oof_mu, oof_sig_raw, oof_abs_wsb, models_mu, models_sigma, sigma_params, oof_score = train_mu_sigma_oof(train, folds)

# Train final models on full data (same features, no target-derived) 
def build_full_features(df):
    # use all available rows to compute leak-safe time proxies (Weeks known)
    basew = df.groupby(PID)[WEEK].min().rename('baseline_week')
    # order/counts
    visit_index = df.groupby(PID)[WEEK].rank(method='first', ascending=True).astype(int)
    visits_count = df.groupby(PID)[WEEK].transform('count')
    visits_remaining = (visits_count - visit_index).astype(int)
    feats = pd.DataFrame({PID: df[PID].values, WEEK: df[WEEK].values})
    feats = feats.join(basew, on=PID)
    feats['weeks_since_baseline'] = feats[WEEK] - feats['baseline_week']
    feats['wsb_sq'] = feats['weeks_since_baseline']**2
    feats['abs_wsb'] = feats['weeks_since_baseline'].abs()
    feats['log_abs_wsb'] = np.log1p(feats['abs_wsb'].values)
    feats['visit_index'] = visit_index.values
    feats['visits_remaining'] = visits_remaining.values
    feats['mean_percent'] = df.groupby(PID)['Percent'].transform('mean').values
    feats['wsb_x_percent'] = feats['weeks_since_baseline'].values * df['Percent'].values
    sex_num = (df['Sex'].values == 'Male').astype(int)
    feats['age_x_sex'] = df['Age'].values * sex_num
    feats['percent_x_age'] = df['Percent'].values * df['Age'].values
    feats['sex_num'] = sex_num
    return feats[['baseline_week','weeks_since_baseline','wsb_sq','abs_wsb','log_abs_wsb','visit_index','visits_remaining','mean_percent','wsb_x_percent','age_x_sex','percent_x_age','sex_num']]

feats_full = build_full_features(train)
X_full = pd.concat([feats_full.reset_index(drop=True), train[NUM_COLS + CAT_COLS].reset_index(drop=True)], axis=1)
y_full = train[TARGET].values
task_type = 'GPU'
try:
    cb.CatBoostRegressor(task_type=task_type)
except Exception:
    task_type = 'CPU'
final_mu = cb.CatBoostRegressor(depth=6, learning_rate=0.06, iterations=4000, l2_leaf_reg=8.0, loss_function='Quantile:alpha=0.5',
                                 bootstrap_type='Bernoulli', subsample=0.8,
                                 random_seed=42, early_stopping_rounds=200, task_type=task_type, verbose=200, allow_writing_files=False)
cat_idx_full = cat_features_indices(X_full, CAT_COLS)
# Keep and reuse exact training column order for inference
X_cols = list(X_full.columns)
print('Train columns:', X_cols)
print('Cat idx full:', cat_idx_full)
final_mu.fit(X_full, y_full, cat_features=cat_idx_full, verbose=200)
res_full = np.abs(y_full - final_mu.predict(X_full))
final_sigma = cb.CatBoostRegressor(depth=5, learning_rate=0.05, iterations=1200, l2_leaf_reg=6.0, loss_function='RMSE',
                                   bootstrap_type='Bernoulli', subsample=0.9,
                                   random_seed=42, early_stopping_rounds=100, task_type=task_type, verbose=200, allow_writing_files=False)
final_sigma.fit(X_full, res_full, cat_features=cat_idx_full, verbose=200)

# Prepare submission predictions
ss = pd.read_csv('sample_submission.csv')
sub = ss.copy()
sub[[PID,WEEK]] = sub['Patient_Week'].str.rsplit('_', n=1, expand=True)
sub[WEEK] = sub[WEEK].astype(int)

# Build a lookup of patient attributes from train+test (without using FVC as a feature).
meta_cols = [PID, WEEK] + NUM_COLS + CAT_COLS
meta_train = train[meta_cols].copy()
meta_test = test[meta_cols].copy()
meta_all = pd.concat([meta_train, meta_test], axis=0, ignore_index=True)
meta_all = meta_all.sort_values([PID, WEEK])
fill_cols = NUM_COLS + CAT_COLS
meta_all[fill_cols] = meta_all.groupby(PID)[fill_cols].ffill()
meta_all[fill_cols] = meta_all.groupby(PID)[fill_cols].bfill()

# Join demographics onto submission rows
sub = sub.merge(meta_all.drop(columns=[WEEK]).drop_duplicates(PID), on=PID, how='left')

# Build submission features using train+test combined metadata (Weeks known)
basew_all = meta_all.groupby(PID)[WEEK].min().rename('baseline_week')
sub = sub.join(basew_all, on=PID)
sub['weeks_since_baseline'] = sub[WEEK] - sub['baseline_week']
sub['wsb_sq'] = sub['weeks_since_baseline']**2
sub['abs_wsb'] = sub['weeks_since_baseline'].abs()
sub['log_abs_wsb'] = np.log1p(sub['abs_wsb'].values)
# visit indices within submission directly (avoid missing keys from meta_all)
sub = sub.sort_values([PID, WEEK]).reset_index(drop=True)
sub['visit_index'] = sub.groupby(PID)[WEEK].rank(method='first', ascending=True).astype(int)
sub['visits_count'] = sub.groupby(PID)[WEEK].transform('count').astype(int)
sub['visits_remaining'] = sub['visits_count'] - sub['visit_index']
sub.drop(columns=['visits_count'], inplace=True)
sub['mean_percent'] = sub.groupby(PID)['Percent'].transform('mean')
sub['wsb_x_percent'] = sub['weeks_since_baseline'] * sub['Percent']
sub['sex_num'] = (sub['Sex'] == 'Male').astype(int)
sub['age_x_sex'] = sub['Age'] * sub['sex_num']
sub['percent_x_age'] = sub['Percent'] * sub['Age']

# Reorder columns to match training order and use Pool with cat feature indices
X_sub = sub[X_cols].copy()
pool_sub = cb.Pool(X_sub, cat_features=cat_idx_full)
mu_pred = final_mu.predict(pool_sub)
sigma_pred_raw = final_sigma.predict(pool_sub)
sigma_pred_raw = np.maximum(sigma_pred_raw, 5.0)
s_best, a_best, b_best = sigma_params
sigma_pred = np.maximum(sigma_pred_raw * s_best, a_best + b_best * X_sub['abs_wsb'].values)
sigma_pred = np.maximum(sigma_pred, 70.0)

out = ss.copy()
out['FVC'] = mu_pred.astype(float)
out['Confidence'] = sigma_pred.astype(float)
out.to_csv('submission.csv', index=False)
print('Saved submission.csv')
print('Temporal OOF Laplace:', oof_score, 'Sigma params:', sigma_params)

0:	learn: 300.0699472	test: 391.8282081	best: 391.8282081 (0)	total: 12ms	remaining: 48.2s


200:	learn: 298.2441266	test: 390.2378103	best: 390.2378103 (200)	total: 2.17s	remaining: 41.1s


400:	learn: 296.3266075	test: 388.5721410	best: 388.5721410 (400)	total: 4.44s	remaining: 39.9s


600:	learn: 294.4124607	test: 386.8802637	best: 386.8802637 (600)	total: 6.72s	remaining: 38s


800:	learn: 292.5070256	test: 385.2060339	best: 385.2060339 (800)	total: 9.01s	remaining: 36s


...

400:	learn: 120.8812690	total: 3.6s	remaining: 7.17s


600:	learn: 105.2747039	total: 5.32s	remaining: 5.3s


800:	learn: 94.1576222	total: 7.08s	remaining: 3.53s


1000:	learn: 87.6614027	total: 8.81s	remaining: 1.75s


1199:	learn: 84.1065117	total: 10.6s	remaining: 0us
Saved submission.csv
Temporal OOF Laplace: -8.180996468548859 Sigma params: (1.4, 0.0, 0.3)


In [11]:
# Pivot to within-patient temporal CV with per-patient Huber baseline, blending, and strengthened sigma
import numpy as np, pandas as pd, time, pickle, sys, subprocess as sp
from sklearn.linear_model import HuberRegressor

try:
    import catboost as cb
except Exception:
    sp.run([sys.executable, '-m', 'pip', 'install', 'catboost==1.2.5'], check=True)
    import catboost as cb

PID, WEEK, TARGET = 'Patient','Weeks','FVC'
CAT_COLS = ['Sex','SmokingStatus']
NUM_COLS = ['Percent','Age']

if 'train' not in globals():
    train = pd.read_csv('train.csv')

def build_patient_groups(df, n_splits=5, seed=42):
    pids = df[PID].unique()
    rng = np.random.default_rng(seed)
    rng.shuffle(pids)
    groups = np.array_split(pids, n_splits)
    return [list(g) for g in groups]

def last_k_indices_per_patient(df, k=2):
    idx = []
    for pid, g in df.groupby(PID):
        g_sorted = g.sort_values(WEEK)
        idx.extend(g_sorted.index[-k:].tolist())
    return set(idx)

def huber_slope_intercept(prior_weeks, prior_fvc):
    n = len(prior_weeks)
    if n >= 2:
        X = np.asarray(prior_weeks, dtype=float).reshape(-1,1)
        y = np.asarray(prior_fvc, dtype=float)
        try:
            model = HuberRegressor(epsilon=1.35, alpha=0.0, fit_intercept=True, max_iter=200).fit(X, y)
            slope = float(model.coef_[0]); intercept = float(model.intercept_)
            yhat = model.predict(X); rmse = float(np.sqrt(np.mean((y - yhat)**2)))
        except Exception:
            # fallback to OLS
            x = X[:,0]; y = y
            x_mean, y_mean = x.mean(), y.mean()
            sxx = np.sum((x-x_mean)**2) + 1e-6
            sxy = np.sum((x-x_mean)*(y-y_mean))
            slope = sxy / sxx; intercept = y_mean - slope * x_mean
            rmse = float(np.sqrt(np.mean((y - (slope*x + intercept))**2)))
        return slope, intercept, rmse
    elif n == 1:
        return 0.0, float(prior_fvc[0]), 120.0
    else:
        return 0.0, np.nan, 120.0

def build_temporal_features_for_fold(tr_df, va_df):
    # Precompute safe time proxies (Weeks only): baseline_week per patient from all rows (train set only)
    basew_map = tr_df.groupby(PID)[WEEK].min()
    # Build history dicts from training part only
    hist = {}  # pid -> list of tuples (week, fvc, percent)
    for pid, g in tr_df.groupby(PID):
        gg = g.sort_values(WEEK)
        hist[pid] = list(zip(gg[WEEK].tolist(), gg[TARGET].tolist(), gg['Percent'].tolist()))

    def build_one_side(df_side, use_training_history_only):
        rows = []
        for pid, g in df_side.groupby(PID):
            g_sorted = g.sort_values(WEEK)
            prior_list = hist.get(pid, []) if use_training_history_only else []
            for idx, r in g_sorted.iterrows():
                w = float(r[WEEK]); pct = float(r['Percent']); age = float(r['Age'])
                sex = r['Sex']; smoke = r['SmokingStatus']
                # Use only prior visits with week < current
                prior_weeks = [pw for (pw, pf, pp) in prior_list if pw < w]
                prior_fvc = [pf for (pw, pf, pp) in prior_list if pw < w]
                prior_pct = [pp for (pw, pf, pp) in prior_list if pw < w]
                slope, intercept, prmse = huber_slope_intercept(prior_weeks, prior_fvc)
                mu_lin = (intercept + slope * w) if np.isfinite(intercept) else np.nan
                vcount = len(prior_weeks)
                basew = float(basew_map.get(pid, np.nan))
                wsb = w - basew if np.isfinite(basew) else np.nan
                abs_wsb = abs(wsb) if np.isfinite(wsb) else np.nan
                # short-horizon dynamics
                if vcount > 0:
                    last_w = prior_weeks[-1]
                    last_pct = prior_pct[-1]
                else:
                    last_w = basew if np.isfinite(basew) else w
                    last_pct = pct
                weeks_since_last = w - last_w if np.isfinite(w) and np.isfinite(last_w) else np.nan
                delta_percent_last = pct - last_pct if np.isfinite(pct) and np.isfinite(last_pct) else 0.0
                mean_pct_prior = (np.mean(prior_pct) if len(prior_pct) > 0 else np.nan)
                sex_num = 1 if sex == 'Male' else 0
                rows.append({
                    'idx': idx,
                    'Patient': pid,
                    'Weeks': w,
                    'baseline_week': basew,
                    'weeks_since_baseline': wsb,
                    'abs_wsb': abs_wsb,
                    'log_abs_wsb': np.log1p(abs_wsb) if np.isfinite(abs_wsb) else np.nan,
                    'visit_count_so_far': vcount,
                    'visit_index_so_far': vcount,
                    'mean_percent_so_far': mean_pct_prior,
                    'slope_prior': slope,
                    'intercept_prior': intercept,
                    'mu_lin': mu_lin,
                    'patient_RMSE': prmse,
                    'weeks_since_last': weeks_since_last,
                    'delta_percent_last': delta_percent_last,
                    'wsb_x_percent': (wsb * pct) if np.isfinite(wsb) else np.nan,
                    'age_x_sex': age * sex_num,
                    'percent_x_age': pct * age,
                    'sex_num': sex_num,
                    'Percent': pct, 'Age': age, 'Sex': sex, 'SmokingStatus': smoke
                })
        feats = pd.DataFrame(rows).set_index('idx').sort_index()
        return feats

    X_tr = build_one_side(tr_df, use_training_history_only=True)
    X_va = build_one_side(va_df, use_training_history_only=True)
    return X_tr, X_va

def temporal_cv_with_blend_and_sigma(train, n_splits=5, seed=42, temporal_k=2):
    t0 = time.time()
    train_sorted = train.sort_values([PID, WEEK]).reset_index(drop=False).rename(columns={'index':'orig_idx'})
    patient_groups = build_patient_groups(train_sorted, n_splits=n_splits, seed=seed)
    n = len(train_sorted)
    oof_mu_cat = np.full(n, np.nan, dtype=float)
    oof_mu_lin = np.full(n, np.nan, dtype=float)
    oof_sigma_raw = np.full(n, np.nan, dtype=float)
    oof_abs_wsb = np.full(n, np.nan, dtype=float)
    oof_prmse = np.full(n, np.nan, dtype=float)
    cat_models, sig_models = [], []
    for fi in range(n_splits):
        val_pids = set(patient_groups[fi])
        va_mask_pat = train_sorted[PID].isin(val_pids)
        # Validation rows: last-2 per patient among val patients
        va_rows = train_sorted[va_mask_pat]
        va_last2_idx = last_k_indices_per_patient(va_rows.set_index('orig_idx'), k=temporal_k)
        va_mask = train_sorted['orig_idx'].isin(va_last2_idx)
        tr_mask = ~va_mask
        tr_df = train_sorted.loc[tr_mask, [PID, WEEK, TARGET, 'Percent', 'Age', 'Sex', 'SmokingStatus']].copy()
        va_df = train_sorted.loc[va_mask, [PID, WEEK, TARGET, 'Percent', 'Age', 'Sex', 'SmokingStatus']].copy()
        # Build causal features using only training history for both sides
        X_tr, X_va = build_temporal_features_for_fold(tr_df, va_df)
        # Drop ID/time columns from features before CatBoost
        X_tr = X_tr.drop(columns=[PID, WEEK], errors='ignore')
        X_va = X_va.drop(columns=[PID, WEEK], errors='ignore')
        y_tr, y_va = tr_df[TARGET].values, va_df[TARGET].values
        # CatBoost mu (median)
        task_type = 'GPU'
        try:
            cb.CatBoostRegressor(task_type=task_type)
        except Exception:
            task_type = 'CPU'
        mu_model = cb.CatBoostRegressor(depth=6, learning_rate=0.06, iterations=3000, l2_leaf_reg=8.0,
                                        loss_function='Quantile:alpha=0.5', bootstrap_type='Bernoulli', subsample=0.8,
                                        random_seed=seed, early_stopping_rounds=200, task_type=task_type, verbose=200, allow_writing_files=False)
        cat_idx = [X_tr.columns.get_loc(c) for c in CAT_COLS if c in X_tr.columns]
        mu_model.fit(X_tr, y_tr, eval_set=(X_va, y_va), cat_features=cat_idx, verbose=200)
        mu_va_cat = mu_model.predict(X_va)
        # Linear baseline mu on val (already computed in features)
        mu_va_lin = X_va['mu_lin'].values
        oof_mu_cat[va_mask.values] = mu_va_cat
        oof_mu_lin[va_mask.values] = mu_va_lin
        # sigma raw model on |residual_cat|
        res_tr = np.abs(y_tr - mu_model.predict(X_tr))
        sig_model = cb.CatBoostRegressor(depth=5, learning_rate=0.05, iterations=1200, l2_leaf_reg=6.0, loss_function='RMSE',
                                         bootstrap_type='Bernoulli', subsample=0.9,
                                         random_seed=seed, early_stopping_rounds=100, task_type=task_type, verbose=200, allow_writing_files=False)
        sig_model.fit(X_tr, res_tr, eval_set=(X_va, np.abs(y_va - mu_va_cat)), cat_features=cat_idx, verbose=200)
        sig_va = np.maximum(sig_model.predict(X_va), 5.0)
        oof_sigma_raw[va_mask.values] = sig_va
        oof_abs_wsb[va_mask.values] = np.asarray(X_va['abs_wsb'].values, dtype=float)
        oof_prmse[va_mask.values] = np.asarray(X_va['patient_RMSE'].values, dtype=float)
        cat_models.append(mu_model); sig_models.append(sig_model)
        print(f"Fold {fi} done. tr={tr_mask.sum()} va={va_mask.sum()}")
    # Choose blend weight on last-2 OOF
    y_oof = train_sorted[TARGET].values
    valid_mask = ~np.isnan(oof_mu_cat) & ~np.isnan(oof_mu_lin)
    ws = [0.3, 0.5, 0.7]
    best_w, best_ll = 0.5, -1e9
    for w in ws:
        mu_blend = w * oof_mu_cat[valid_mask] + (1-w) * oof_mu_lin[valid_mask]
        sigma_tmp = np.maximum(oof_sigma_raw[valid_mask], 70.0)  # placeholder
        scr = laplace_log_likelihood(y_oof[valid_mask], mu_blend, sigma_tmp, sigma_floor=70.0, error_clip=1000.0)
        if scr > best_ll:
            best_ll, best_w = scr, w
    print(f"Blend weight selected: w={best_w} (CatBoost weight)")
    # Sigma calibration with patient_RMSE and |wsb|
    scale_grid = [1.0, 1.2, 1.4]
    a_grid = [0, 20, 40]
    b_grid = [0.3, 0.6, 1.0]
    c_grid = [0.8, 1.0, 1.2]
    best_params, best_score = (1.0, 0.0, 0.6, 1.0), -1e9
    for s in scale_grid:
        for a in a_grid:
            for b in b_grid:
                for c in c_grid:
                    mu_blend = best_w * oof_mu_cat[valid_mask] + (1-best_w) * oof_mu_lin[valid_mask]
                    sig = np.maximum(oof_sigma_raw[valid_mask] * s, a + b * oof_abs_wsb[valid_mask])
                    sig = np.maximum(sig, c * oof_prmse[valid_mask])
                    sig = np.maximum(sig, 70.0)
                    scr = laplace_log_likelihood(y_oof[valid_mask], mu_blend, sig, sigma_floor=70.0, error_clip=1000.0)
                    if scr > best_score:
                        best_score, best_params = scr, (s, a, b, c)
    print(f"Temporal OOF (last-2) with blend+sigma: {best_score:.5f} @ params {best_params}")
    return dict(best_w=best_w, sigma_params=best_params, oof_score=best_score,
                oof_masks=valid_mask, oof_mu_cat=oof_mu_cat, oof_mu_lin=oof_mu_lin,
                oof_sigma_raw=oof_sigma_raw, oof_abs_wsb=oof_abs_wsb, oof_prmse=oof_prmse,
                train_sorted=train_sorted, models=(cat_models, sig_models))

# Run the new temporal CV pipeline (this does NOT overwrite previous submission yet)
res = temporal_cv_with_blend_and_sigma(train, n_splits=5, seed=42, temporal_k=2)
print('Done temporal CV pivot. Best OOF:', res['oof_score'])

0:	learn: 319.3757989	test: 282.3315430	best: 282.3315430 (0)	total: 12.2ms	remaining: 36.5s


200:	learn: 317.2689145	test: 280.0167847	best: 280.0167847 (200)	total: 2.23s	remaining: 31.1s


400:	learn: 315.1165648	test: 277.6965332	best: 277.6965332 (400)	total: 4.54s	remaining: 29.4s


600:	learn: 312.9640273	test: 275.4343872	best: 275.4343872 (600)	total: 6.85s	remaining: 27.3s


800:	learn: 310.8187030	test: 273.1909180	best: 273.1909180 (800)	total: 9.13s	remaining: 25.1s


...

400:	learn: 138.0437325	test: 208.1301159	best: 207.7699954 (384)	total: 3.69s	remaining: 7.36s


600:	learn: 131.4667087	test: 205.8275194	best: 205.5806490 (599)	total: 5.21s	remaining: 5.19s


800:	learn: 121.0873472	test: 196.6488191	best: 196.5470374 (799)	total: 7.04s	remaining: 3.51s


1000:	learn: 112.4033385	test: 189.8557390	best: 189.7825687 (972)	total: 8.91s	remaining: 1.77s


1199:	learn: 107.0530832	test: 187.6230994	best: 187.4431957 (1174)	total: 10.7s	remaining: 0us
bestTest = 187.4431957
bestIteration = 1174
Shrink model to first 1175 iterations.
Fold 4 done. tr=1332 va=62
Blend weight selected: w=0.3 (CatBoost weight)
Temporal OOF (last-2) with blend+sigma: -7.51715 @ params (1.0, 0, 0.3, 1.2)
Done temporal CV pivot. Best OOF: -7.517147066784831


In [12]:
# Final training on full data with causal features; build submission using prior train history only
import pandas as pd, numpy as np, time

PID, WEEK, TARGET = 'Patient','Weeks','FVC'
CAT_COLS = ['Sex','SmokingStatus']
NUM_COLS = ['Percent','Age']

assert 'res' in globals(), 'Run the temporal CV cell to obtain best weights/params (res dict).'
best_w = res['best_w']
s_best, a_best, b_best, c_best = res['sigma_params']
print('Using blend weight (CatBoost):', best_w, 'Sigma params:', (s_best, a_best, b_best, c_best))

try:
    import catboost as cb
except Exception:
    import sys, subprocess as sp
    sp.run([sys.executable, '-m', 'pip', 'install', 'catboost==1.2.5'], check=True)
    import catboost as cb

from sklearn.linear_model import HuberRegressor

# Reuse feature builders from Cell 4, but generalize for full-data train and submission
def huber_slope_intercept(prior_weeks, prior_fvc):
    n = len(prior_weeks)
    if n >= 2:
        X = np.asarray(prior_weeks, dtype=float).reshape(-1,1)
        y = np.asarray(prior_fvc, dtype=float)
        try:
            model = HuberRegressor(epsilon=1.35, alpha=0.0, fit_intercept=True, max_iter=200).fit(X, y)
            slope = float(model.coef_[0]); intercept = float(model.intercept_)
            yhat = model.predict(X); rmse = float(np.sqrt(np.mean((y - yhat)**2)))
        except Exception:
            x = X[:,0]; y = y
            x_mean, y_mean = x.mean(), y.mean()
            sxx = np.sum((x-x_mean)**2) + 1e-6
            sxy = np.sum((x-x_mean)*(y-y_mean))
            slope = sxy / sxx; intercept = y_mean - slope * x_mean
            rmse = float(np.sqrt(np.mean((y - (slope*x + intercept))**2)))
        return slope, intercept, rmse
    elif n == 1:
        return 0.0, float(prior_fvc[0]), 120.0
    else:
        return 0.0, np.nan, 120.0

def build_history_from_train(train_df):
    hist = {}
    for pid, g in train_df.groupby(PID):
        gg = g.sort_values(WEEK)
        hist[pid] = list(zip(gg[WEEK].tolist(), gg[TARGET].tolist(), gg['Percent'].tolist()))
    return hist

def build_features_given_history(df_target, hist, basew_map):
    rows = []
    for pid, g in df_target.groupby(PID):
        g_sorted = g.sort_values(WEEK)
        prior_list = hist.get(pid, [])
        for idx, r in g_sorted.iterrows():
            w = float(r[WEEK]); pct = float(r['Percent']); age = float(r['Age'])
            sex = r['Sex']; smoke = r['SmokingStatus']
            prior_weeks = [pw for (pw, pf, pp) in prior_list if pw < w]
            prior_fvc = [pf for (pw, pf, pp) in prior_list if pw < w]
            prior_pct = [pp for (pw, pf, pp) in prior_list if pw < w]
            slope, intercept, prmse = huber_slope_intercept(prior_weeks, prior_fvc)
            mu_lin = (intercept + slope * w) if np.isfinite(intercept) else np.nan
            vcount = len(prior_weeks)
            basew = float(basew_map.get(pid, np.nan))
            wsb = w - basew if np.isfinite(basew) else np.nan
            abs_wsb = abs(wsb) if np.isfinite(wsb) else np.nan
            sex_num = 1 if sex == 'Male' else 0
            rows.append({
                'idx': idx, 'Patient': pid, 'Weeks': w,
                'baseline_week': basew, 'weeks_since_baseline': wsb,
                'abs_wsb': abs_wsb, 'log_abs_wsb': np.log1p(abs_wsb) if np.isfinite(abs_wsb) else np.nan,
                'visit_count_so_far': vcount, 'visit_index_so_far': vcount,
                'mean_percent_so_far': (np.mean(prior_pct) if vcount > 0 else np.nan),
                'slope_prior': slope, 'intercept_prior': intercept, 'mu_lin': mu_lin, 'patient_RMSE': prmse,
                'wsb_x_percent': (wsb * pct) if np.isfinite(wsb) else np.nan,
                'age_x_sex': age * sex_num, 'percent_x_age': pct * age, 'sex_num': sex_num,
                'Percent': pct, 'Age': age, 'Sex': sex, 'SmokingStatus': smoke
            })
    feats = pd.DataFrame(rows).set_index('idx').sort_index()
    return feats

# Prepare full train sorted
train_sorted = train.sort_values([PID, WEEK]).reset_index(drop=True)
basew_map_full = train_sorted.groupby(PID)[WEEK].min()
hist_train = build_history_from_train(train_sorted)
X_train_full = build_features_given_history(train_sorted[[PID, WEEK, TARGET, 'Percent','Age','Sex','SmokingStatus']], hist_train, basew_map_full)
y_full = train_sorted[TARGET].values
X_train_full_cb = X_train_full.drop(columns=[PID, WEEK], errors='ignore')

# Train CatBoost mu and sigma on full causal features
task_type = 'GPU'
try:
    cb.CatBoostRegressor(task_type=task_type)
except Exception:
    task_type = 'CPU'
mu_full = cb.CatBoostRegressor(depth=6, learning_rate=0.06, iterations=4000, l2_leaf_reg=8.0,
                                loss_function='Quantile:alpha=0.5', bootstrap_type='Bernoulli', subsample=0.8,
                                random_seed=42, early_stopping_rounds=200, task_type=task_type, verbose=200, allow_writing_files=False)
cat_idx_full = [X_train_full_cb.columns.get_loc(c) for c in CAT_COLS if c in X_train_full_cb.columns]
print('Train columns (full temporal):', list(X_train_full_cb.columns))
print('Cat idx:', cat_idx_full)
mu_full.fit(X_train_full_cb, y_full, cat_features=cat_idx_full, verbose=200)
res_full = np.abs(y_full - mu_full.predict(X_train_full_cb))
sig_full = cb.CatBoostRegressor(depth=5, learning_rate=0.05, iterations=1200, l2_leaf_reg=6.0, loss_function='RMSE',
                                bootstrap_type='Bernoulli', subsample=0.9,
                                random_seed=42, early_stopping_rounds=100, task_type=task_type, verbose=200, allow_writing_files=False)
sig_full.fit(X_train_full_cb, res_full, cat_features=cat_idx_full, verbose=200)

# Build submission rows with demographics from train+test (no FVC from test) and features using train history only
ss = pd.read_csv('sample_submission.csv')
sub = ss.copy()
sub[[PID, WEEK]] = sub['Patient_Week'].str.rsplit('_', n=1, expand=True)
sub[WEEK] = sub[WEEK].astype(int)
meta_cols = [PID, WEEK] + NUM_COLS + CAT_COLS
meta_train = train[meta_cols].copy()
meta_test = pd.read_csv('test.csv')[meta_cols].copy()
meta_all = pd.concat([meta_train, meta_test], axis=0, ignore_index=True).sort_values([PID, WEEK])
fill_cols = NUM_COLS + CAT_COLS
meta_all[fill_cols] = meta_all.groupby(PID)[fill_cols].ffill()
meta_all[fill_cols] = meta_all.groupby(PID)[fill_cols].bfill()
sub = sub.merge(meta_all.drop(columns=[WEEK]).drop_duplicates(PID), on=PID, how='left')

# Features for submission using train history only
X_sub_feats = build_features_given_history(sub[[PID, WEEK] + NUM_COLS + CAT_COLS], hist_train, basew_map_full)
X_sub_cb = X_sub_feats.drop(columns=[PID, WEEK], errors='ignore')
pool_sub = cb.Pool(X_sub_cb, cat_features=cat_idx_full)
mu_cat_sub = mu_full.predict(pool_sub)
mu_lin_sub = X_sub_feats['mu_lin'].values
mu_final = best_w * mu_cat_sub + (1 - best_w) * mu_lin_sub
sigma_raw_sub = np.maximum(sig_full.predict(pool_sub), 5.0)
abs_wsb_sub = X_sub_feats['abs_wsb'].values.astype(float)
prmse_sub = X_sub_feats['patient_RMSE'].values.astype(float)
sigma_cal = np.maximum(sigma_raw_sub * s_best, a_best + b_best * abs_wsb_sub)
sigma_cal = np.maximum(sigma_cal, c_best * prmse_sub)
sigma_cal = np.maximum(sigma_cal, 70.0)

out = ss.copy()
out['FVC'] = mu_final.astype(float)
out['Confidence'] = sigma_cal.astype(float)
out.to_csv('submission.csv', index=False)
print('Saved submission.csv with temporal features + blend')

Using blend weight (CatBoost): 0.3 Sigma params: (1.0, 0, 0.3, 1.2)


Train columns (full temporal): ['baseline_week', 'weeks_since_baseline', 'abs_wsb', 'log_abs_wsb', 'visit_count_so_far', 'visit_index_so_far', 'mean_percent_so_far', 'slope_prior', 'intercept_prior', 'mu_lin', 'patient_RMSE', 'wsb_x_percent', 'age_x_sex', 'percent_x_age', 'sex_num', 'Percent', 'Age', 'Sex', 'SmokingStatus']
Cat idx: [17, 18]
0:	learn: 317.6557120	total: 12.6ms	remaining: 50.3s


200:	learn: 315.5264527	total: 2.11s	remaining: 39.9s


400:	learn: 313.3566625	total: 4.38s	remaining: 39.3s


600:	learn: 311.1940907	total: 6.63s	remaining: 37.5s


...

400:	learn: 142.6307601	total: 3.62s	remaining: 7.22s


600:	learn: 128.3657179	total: 5.41s	remaining: 5.39s


800:	learn: 119.4171282	total: 7.21s	remaining: 3.59s


1000:	learn: 113.8653084	total: 8.89s	remaining: 1.77s


1199:	learn: 108.4907648	total: 10.6s	remaining: 0us
Saved submission.csv with temporal features + blend


In [28]:
# Temporal CV v2: add LGBM quantiles with residualization, blend mu (residual heads), blend sigma_q with Cat sigma, calibrate
import numpy as np, pandas as pd, time, sys, subprocess as sp

try:
    import lightgbm as lgb
except Exception:
    sp.run([sys.executable, '-m', 'pip', 'install', 'lightgbm==4.3.0'], check=True)
    import lightgbm as lgb

try:
    import catboost as cb
except Exception:
    sp.run([sys.executable, '-m', 'pip', 'install', 'catboost==1.2.5'], check=True)
    import catboost as cb

from sklearn.linear_model import HuberRegressor

PID, WEEK, TARGET = 'Patient','Weeks','FVC'
CAT_COLS = ['Sex','SmokingStatus']

if 'train' not in globals():
    train = pd.read_csv('train.csv')

# Reuse helpers from Cell 4 (huber_slope_intercept, build_patient_groups, last_k_indices_per_patient, build_temporal_features_for_fold)

def temporal_cv_with_lgbm_and_residuals(train, n_splits=5, seed=42, temporal_k=2, seeds_bag=(42,123,456,789,2024)):
    train_sorted = train.sort_values([PID, WEEK]).reset_index(drop=False).rename(columns={'index':'orig_idx'})
    patient_groups = build_patient_groups(train_sorted, n_splits=n_splits, seed=seed)
    n = len(train_sorted)
    # OOF accumulators for seed-bagging
    bag_oof_mu_lin = np.zeros(n, dtype=float)
    bag_oof_res_cat = np.zeros(n, dtype=float)
    bag_oof_res_lgbm = np.zeros(n, dtype=float)
    bag_oof_sigma_cat_raw = np.zeros(n, dtype=float)
    bag_oof_sigma_q = np.zeros(n, dtype=float)
    bag_oof_abs_wsb = np.zeros(n, dtype=float)
    bag_oof_prmse = np.zeros(n, dtype=float)
    bag_oof_wsl = np.zeros(n, dtype=float)
    bag_oof_vcount = np.zeros(n, dtype=float)
    bag_counts = np.zeros(n, dtype=float)

    global_med = float(train[TARGET].median())

    # For returning last seed's models (optional, not used for OOF)
    cat_models, cat_sig_models = [], []
    lgb_models_q15, lgb_models_q50, lgb_models_q85 = [], [], []

    for sdx, s_seed in enumerate(seeds_bag):
        print(f"Seed bag {sdx+1}/{len(seeds_bag)} (seed={s_seed})", flush=True)
        # Per-seed OOF holders
        oof_mu_lin = np.full(n, np.nan, dtype=float)
        oof_res_cat = np.full(n, np.nan, dtype=float)
        oof_res_lgbm = np.full(n, np.nan, dtype=float)
        oof_sigma_cat_raw = np.full(n, np.nan, dtype=float)
        oof_sigma_q = np.full(n, np.nan, dtype=float)
        oof_abs_wsb = np.full(n, np.nan, dtype=float)
        oof_prmse = np.full(n, np.nan, dtype=float)
        oof_wsl = np.full(n, np.nan, dtype=float)
        oof_vcount = np.full(n, np.nan, dtype=float)

        for fi in range(n_splits):
            val_pids = set(patient_groups[fi])
            va_mask_pat = train_sorted[PID].isin(val_pids)
            va_rows = train_sorted[va_mask_pat]
            va_last2_idx = last_k_indices_per_patient(va_rows.set_index('orig_idx'), k=temporal_k)
            va_mask = train_sorted['orig_idx'].isin(va_last2_idx)
            tr_mask = ~va_mask
            tr_df = train_sorted.loc[tr_mask, [PID, WEEK, TARGET, 'Percent', 'Age', 'Sex', 'SmokingStatus']].copy()
            va_df = train_sorted.loc[va_mask, [PID, WEEK, TARGET, 'Percent', 'Age', 'Sex', 'SmokingStatus']].copy()

            # Causal features from training history only (with cold-start-safe basew/wsl inside builder in Cell 4)
            X_tr_full, X_va_full = build_temporal_features_for_fold(tr_df, va_df)
            # Cold-start safety: ensure baseline_week/abs_wsb/weeks_since_last sane
            for X_ in (X_tr_full, X_va_full):
                # baseline_week fallback to current week if missing
                miss_base = ~np.isfinite(X_['baseline_week'].values)
                if miss_base.any():
                    X_.loc[miss_base, 'baseline_week'] = X_.loc[miss_base, WEEK].values
                # recompute derived safely
                X_['weeks_since_baseline'] = X_[WEEK] - X_['baseline_week']
                X_['abs_wsb'] = np.abs(X_['weeks_since_baseline'])
                # weeks_since_last: if NaN, set 0.0 (first visit or unknown last_w)
                if 'weeks_since_last' in X_.columns:
                    X_['weeks_since_last'] = np.nan_to_num(X_['weeks_since_last'].values, nan=0.0, posinf=0.0, neginf=0.0)

            mu_lin_tr_raw = X_tr_full['mu_lin'].values
            mu_lin_va_raw = X_va_full['mu_lin'].values
            int_tr = X_tr_full['intercept_prior'].values
            int_va = X_va_full['intercept_prior'].values
            oof_abs_wsb[va_mask.values] = X_va_full['abs_wsb'].values.astype(float)
            oof_prmse[va_mask.values] = X_va_full['patient_RMSE'].values.astype(float)
            # propagate weeks_since_last (robust to missing col)
            if 'weeks_since_last' in X_va_full.columns:
                wsl_vals = X_va_full['weeks_since_last'].values.astype(float)
            else:
                wsl_vals = np.full(len(X_va_full), 0.0, dtype=float)
            oof_wsl[va_mask.values] = wsl_vals
            # visit_count_so_far for calibration subset
            if 'visit_count_so_far' in X_va_full.columns:
                oof_vcount[va_mask.values] = X_va_full['visit_count_so_far'].values.astype(float)
            else:
                oof_vcount[va_mask.values] = 0.0

            X_tr = X_tr_full.drop(columns=[PID, WEEK], errors='ignore')
            X_va = X_va_full.drop(columns=[PID, WEEK], errors='ignore')
            y_tr = tr_df[TARGET].values
            y_va = va_df[TARGET].values
            # Leak-free mu_lin fills using intercept_prior or global median
            mu_lin_tr = np.where(np.isfinite(mu_lin_tr_raw), mu_lin_tr_raw,
                                 np.where(np.isfinite(int_tr), int_tr, global_med))
            mu_lin_va = np.where(np.isfinite(mu_lin_va_raw), mu_lin_va_raw,
                                 np.where(np.isfinite(int_va), int_va, global_med))
            oof_mu_lin[va_mask.values] = mu_lin_va
            y_res_tr = y_tr - mu_lin_tr

            # Residual head: drop mu_lin from features
            X_tr_res = X_tr.drop(columns=['mu_lin'], errors='ignore')
            X_va_res = X_va.drop(columns=['mu_lin'], errors='ignore')

            # Add cheap, causal interaction features (+ sqrt_abs_wsb)
            for X in (X_tr_res, X_va_res):
                X['pos_wsb'] = np.clip(X['weeks_since_baseline'], 0, None)
                X['neg_wsb'] = np.clip(-X['weeks_since_baseline'], 0, None)
                X['percent_centered'] = X['Percent'] - X['mean_percent_so_far']
                X['percent_poswsb'] = X['Percent'] * X['pos_wsb']
                X['percent_negwsb'] = X['Percent'] * X['neg_wsb']
                X['wsb_x_age'] = X['weeks_since_baseline'] * X['Age']
                X['has_history'] = (X['visit_count_so_far'] > 0).astype(int)
                X['sqrt_abs_wsb'] = np.sqrt(np.abs(X['weeks_since_baseline']))

            # CatBoost residual head (tweaked params)
            task_type = 'GPU'
            try:
                cb.CatBoostRegressor(task_type=task_type)
            except Exception:
                task_type = 'CPU'
            cat_params = dict(depth=6, learning_rate=0.06, iterations=2500, l2_leaf_reg=8.0,
                               loss_function='Quantile:alpha=0.5', bootstrap_type='Bernoulli', subsample=0.8,
                               random_seed=s_seed, early_stopping_rounds=200, task_type=task_type, verbose=200, allow_writing_files=False)
            if task_type == 'CPU':
                cat_params['colsample_bylevel'] = 0.8
            cat_res = cb.CatBoostRegressor(**cat_params)
            cat_idx = [X_tr_res.columns.get_loc(c) for c in CAT_COLS if c in X_tr_res.columns]
            cat_res.fit(X_tr_res, y_res_tr, eval_set=(X_va_res, y_va - mu_lin_va), cat_features=cat_idx, verbose=200)
            res_cat_va = cat_res.predict(X_va_res)
            oof_res_cat[va_mask.values] = res_cat_va

            # LightGBM quantiles on residual: drop object categorical columns
            X_tr_lgb = X_tr_res.drop(columns=CAT_COLS, errors='ignore')
            X_va_lgb = X_va_res.drop(columns=CAT_COLS, errors='ignore')

            lgb_params = dict(objective='quantile', n_estimators=2000, learning_rate=0.03,
                              num_leaves=31, max_depth=6, subsample=0.8, colsample_bytree=0.8,
                              min_child_samples=10, reg_alpha=0.1, reg_lambda=0.2,
                              verbose=-1, random_state=s_seed, force_row_wise=True)
            lgb_tr = lgb.Dataset(X_tr_lgb, label=y_res_tr, free_raw_data=False)
            lgb_va = lgb.Dataset(X_va_lgb, label=y_va - mu_lin_va, reference=lgb_tr, free_raw_data=False)

            # q15
            p15 = lgb_params.copy(); p15['alpha'] = 0.15
            m15 = lgb.train(p15, lgb_tr, valid_sets=[lgb_va],
                            num_boost_round=p15['n_estimators'],
                            callbacks=[lgb.early_stopping(200, verbose=False), lgb.log_evaluation(0)])
            # q50
            p50 = lgb_params.copy(); p50['alpha'] = 0.50
            m50 = lgb.train(p50, lgb_tr, valid_sets=[lgb_va],
                            num_boost_round=p50['n_estimators'],
                            callbacks=[lgb.early_stopping(200, verbose=False), lgb.log_evaluation(0)])
            # q85
            p85 = lgb_params.copy(); p85['alpha'] = 0.85
            m85 = lgb.train(p85, lgb_tr, valid_sets=[lgb_va],
                            num_boost_round=p85['n_estimators'],
                            callbacks=[lgb.early_stopping(200, verbose=False), lgb.log_evaluation(0)])

            res_q15_va = m15.predict(X_va_lgb, num_iteration=m15.best_iteration)
            res_q50_va = m50.predict(X_va_lgb, num_iteration=m50.best_iteration)
            res_q85_va = m85.predict(X_va_lgb, num_iteration=m85.best_iteration)
            oof_res_lgbm[va_mask.values] = res_q50_va
            # sigma_q from quantile spread (scale fix: divide by 2.0)
            sigma_q_va = (res_q85_va - res_q15_va) / 2.0
            oof_sigma_q[va_mask.values] = np.maximum(sigma_q_va, 5.0)

            # Cat sigma raw on Cat residual head (tweaked params)
            res_tr_cat_pred = cat_res.predict(X_tr_res)
            mu_tr_cat = mu_lin_tr + res_tr_cat_pred
            sig_params = dict(depth=5, learning_rate=0.05, iterations=1200, l2_leaf_reg=6.0, loss_function='RMSE',
                               bootstrap_type='Bernoulli', subsample=0.9,
                               random_seed=s_seed, early_stopping_rounds=100, task_type=task_type, verbose=200, allow_writing_files=False)
            if task_type == 'CPU':
                sig_params['colsample_bylevel'] = 0.8
            sig_cat = cb.CatBoostRegressor(**sig_params)
            sig_cat.fit(X_tr_res, np.abs(y_tr - mu_tr_cat), eval_set=(X_va_res, np.abs(y_va - (mu_lin_va + res_cat_va))),
                        cat_features=cat_idx, verbose=200)
            sig_raw_va = np.maximum(sig_cat.predict(X_va_res), 5.0)
            oof_sigma_cat_raw[va_mask.values] = sig_raw_va

            # Save models of last seed (optional)
            if sdx == len(seeds_bag) - 1:
                cat_models.append(cat_res); cat_sig_models.append(sig_cat)
                lgb_models_q15.append(m15); lgb_models_q50.append(m50); lgb_models_q85.append(m85)
            print(f"Seed {s_seed} Fold {fi} done. tr={tr_mask.sum()} va={va_mask.sum()}")

        # Accumulate this seed's OOF into bag
        valid_seed = ~np.isnan(oof_mu_lin)
        bag_oof_mu_lin[valid_seed] += oof_mu_lin[valid_seed]
        bag_oof_res_cat[valid_seed] += oof_res_cat[valid_seed]
        bag_oof_res_lgbm[valid_seed] += oof_res_lgbm[valid_seed]
        bag_oof_sigma_cat_raw[valid_seed] += oof_sigma_cat_raw[valid_seed]
        bag_oof_sigma_q[valid_seed] += oof_sigma_q[valid_seed]
        bag_oof_abs_wsb[valid_seed] += oof_abs_wsb[valid_seed]
        bag_oof_prmse[valid_seed] += oof_prmse[valid_seed]
        bag_oof_wsl[valid_seed] += oof_wsl[valid_seed]
        bag_oof_vcount[valid_seed] += oof_vcount[valid_seed]
        bag_counts[valid_seed] += 1.0

    # Average across seeds
    nonzero = bag_counts > 0
    oof_mu_lin = np.where(nonzero, bag_oof_mu_lin / np.maximum(bag_counts, 1e-9), np.nan)
    oof_res_cat = np.where(nonzero, bag_oof_res_cat / np.maximum(bag_counts, 1e-9), np.nan)
    oof_res_lgbm = np.where(nonzero, bag_oof_res_lgbm / np.maximum(bag_counts, 1e-9), np.nan)
    oof_sigma_cat_raw = np.where(nonzero, bag_oof_sigma_cat_raw / np.maximum(bag_counts, 1e-9), np.nan)
    oof_sigma_q = np.where(nonzero, bag_oof_sigma_q / np.maximum(bag_counts, 1e-9), np.nan)
    oof_abs_wsb = np.where(nonzero, bag_oof_abs_wsb / np.maximum(bag_counts, 1e-9), np.nan)
    oof_prmse = np.where(nonzero, bag_oof_prmse / np.maximum(bag_counts, 1e-9), np.nan)
    oof_wsl = np.where(nonzero, bag_oof_wsl / np.maximum(bag_counts, 1e-9), np.nan)
    oof_vcount = np.where(nonzero, bag_oof_vcount / np.maximum(bag_counts, 1e-9), np.nan)

    # Tune mu residual blend weight and sigma blend/calibration grids
    valid_mask = ~np.isnan(oof_res_cat) & ~np.isnan(oof_res_lgbm) & ~np.isnan(oof_mu_lin)
    y_all = train_sorted[TARGET].values
    y_oof = y_all[valid_mask]
    mu_lin_oof = oof_mu_lin[valid_mask]
    res_cat_oof = oof_res_cat[valid_mask]
    res_lgbm_oof = oof_res_lgbm[valid_mask]
    abs_wsb_oof = oof_abs_wsb[valid_mask]
    prmse_oof = oof_prmse[valid_mask]
    sig_cat_oof = oof_sigma_cat_raw[valid_mask]
    sig_q_oof = oof_sigma_q[valid_mask]
    wsl_oof = oof_wsl[valid_mask]
    vcount_oof = oof_vcount[valid_mask]

    # NaN safety before grid search
    y_oof = np.nan_to_num(y_oof, nan=np.nanmedian(y_oof))
    mu_lin_oof = np.nan_to_num(mu_lin_oof, nan=np.nanmedian(mu_lin_oof))
    res_cat_oof = np.nan_to_num(res_cat_oof, nan=0.0)
    res_lgbm_oof = np.nan_to_num(res_lgbm_oof, nan=0.0)
    abs_wsb_oof = np.nan_to_num(abs_wsb_oof, nan=0.0, posinf=0.0, neginf=0.0)
    prmse_oof = np.nan_to_num(prmse_oof, nan=np.nanmedian(prmse_oof))
    wsl_oof_clean = np.nan_to_num(wsl_oof, nan=0.0, posinf=0.0, neginf=0.0)
    sig_cat_oof = np.nan_to_num(sig_cat_oof, nan=np.nanmedian(sig_cat_oof))
    sig_q_oof = np.nan_to_num(sig_q_oof, nan=np.nanmedian(sig_q_oof))

    d_grid = [0.0, 0.3, 0.5, 0.7, 1.0]
    s_grid = [1.0, 0.8, 1.2, 1.4, 1.6]
    a_grid = [0, 20, 40, 60]
    b_grid = [0.2, 0.3, 0.4, 0.6, 0.8, 1.0]
    c_grid = [0.8, 1.0, 1.2, 1.4, 1.5]
    e_grid = [0.0, 0.5, 1.0]

    # choose blend weight w_cat on full valid set (sigma placeholder with floor)
    best_w, best_ll = 0.5, -1e9
    for w_cat in [0.3, 0.4, 0.5, 0.6, 0.7]:
        mu_base = mu_lin_oof + w_cat * res_cat_oof + (1 - w_cat) * res_lgbm_oof
        mu_base = np.nan_to_num(mu_base, nan=np.nanmedian(mu_base))
        sig_tmp = np.maximum(np.maximum(sig_cat_oof, 70.0), np.maximum(70.0, 70.0 + 0.0 * np.abs(wsl_oof_clean)))
        mask_ok = np.isfinite(y_oof) & np.isfinite(mu_base) & np.isfinite(sig_tmp)
        if not np.any(mask_ok):
            continue
        scr = laplace_log_likelihood(y_oof[mask_ok], mu_base[mask_ok], sig_tmp[mask_ok], sigma_floor=70.0, error_clip=1000.0)
        if np.isfinite(scr) and scr > best_ll:
            best_ll, best_w = scr, w_cat

    # Cold-start-focused sigma calibration subset: visit_count_so_far <= 2
    calib_mask = (vcount_oof <= 2)
    if not np.any(calib_mask):
        calib_mask = np.ones_like(y_oof, dtype=bool)

    # Diagnostics
    print('Diag valid_count:', valid_mask.sum(),
          ' nan(abs_wsb)=', int(np.isnan(oof_abs_wsb[valid_mask]).sum()),
          ' nan(wsl)=', int(np.isnan(oof_wsl[valid_mask]).sum()),
          ' nan(prmse)=', int(np.isnan(oof_prmse[valid_mask]).sum()),
          ' calib_count=', int(calib_mask.sum()))

    best_score = -1e9
    best = dict(w_cat=best_w, d=None, s=None, a=None, b=None, c=None, e=None)

    for d in d_grid:
        sig_raw = np.maximum(d * sig_q_oof + (1 - d) * sig_cat_oof, 5.0)
        for s in s_grid:
            for a in a_grid:
                for b in b_grid:
                    for c in c_grid:
                        for e in e_grid:
                            sig = np.maximum(sig_raw * s, a + b * np.clip(abs_wsb_oof, 0, 40))
                            sig = np.maximum(sig, c * prmse_oof)
                            sig = np.maximum(sig, 70.0 + e * np.abs(wsl_oof_clean))
                            sig = np.maximum(sig, 70.0)
                            mu_base = best_w * res_cat_oof + (1 - best_w) * res_lgbm_oof + mu_lin_oof
                            mu_base = np.nan_to_num(mu_base, nan=np.nanmedian(mu_base))
                            mask_ok = calib_mask & np.isfinite(y_oof) & np.isfinite(mu_base) & np.isfinite(sig)
                            if not np.any(mask_ok):
                                continue
                            scr = laplace_log_likelihood(y_oof[mask_ok], mu_base[mask_ok], sig[mask_ok], sigma_floor=70.0, error_clip=1000.0)
                            if np.isfinite(scr) and scr > best_score:
                                best_score = scr
                                best.update(dict(d=d, s=s, a=a, b=b, c=c, e=e))

    print(f"Temporal OOF (last-{temporal_k}) residual blend (seed-bagged {len(seeds_bag)}): {best_score:.5f} with params {best}")
    return dict(best_score=best_score, best=best,
                oof_masks=valid_mask,
                oof_mu_lin=oof_mu_lin, oof_res_cat=oof_res_cat, oof_res_lgbm=oof_res_lgbm,
                oof_sigma_cat_raw=oof_sigma_cat_raw, oof_sigma_q=oof_sigma_q,
                oof_abs_wsb=oof_abs_wsb, oof_prmse=oof_prmse, oof_wsl=oof_wsl, oof_vcount=oof_vcount,
                train_sorted=train_sorted,
                models=dict(cat=cat_models, cat_sig=cat_sig_models, lgb_q15=lgb_models_q15, lgb_q50=lgb_models_q50, lgb_q85=lgb_models_q85))

# NOTE: Intentionally do not auto-run the heavy 5-seed pipeline here.
# Use the dedicated diagnostic/prod cells to execute with desired seeds and k.

In [43]:
# Override to LB-oriented conservative parameters (expert primary recommendation after 10-seed last-3 CV)
conservative_params = {'w_cat':0.3,'d':0.5,'s':1.6,'a':50,'b':1.0,'c':1.0,'e':0.7}
orig_train_sorted = res2.get('train_sorted', None) if 'res2' in globals() else None
res2 = {'best': conservative_params, 'train_sorted': orig_train_sorted}
print('Using parameters:', res2['best'])

Using parameters: {'w_cat': 0.3, 'd': 0.5, 's': 1.6, 'a': 50, 'b': 1.0, 'c': 1.0, 'e': 0.7}


In [132]:
# Productionize residual ensemble (CatBoost residual + LGBM quantiles) using best params from res2 with seed-bagging and feature parity
import numpy as np, pandas as pd, sys, subprocess as sp, time, warnings
from scipy.stats import rankdata

assert 'res2' in globals(), 'Run Cell 6 (and optionally 14/15) to obtain best residual-ensemble params (res2 dict).'
best = res2['best']
print('Best params from residual CV:', best)

HEDGE_MODE = 'rankmap_incell_final'

# Expert overrides (medal-safe baseline for mapping)
override = {'w_cat': 0.30, 'd': 0.45, 's': 1.40, 'a': 40.0, 'b': 0.9, 'c': 0.9, 'e': 0.30}
best = {**best, **override}
print('Applied params:', override)

PID, WEEK, TARGET = 'Patient','Weeks','FVC'
CAT_COLS = ['Sex','SmokingStatus']

try:
    import lightgbm as lgb
except Exception:
    sp.run([sys.executable, '-m', 'pip', 'install', 'lightgbm==4.3.0'], check=True)
    import lightgbm as lgb
try:
    import catboost as cb
except Exception:
    sp.run([sys.executable, '-m', 'pip', 'install', 'catboost==1.2.5'], check=True)
    import catboost as cb
from sklearn.linear_model import HuberRegressor

def build_history(df_src):
    hist = {}
    for pid, g in df_src.groupby(PID):
        gg = g.sort_values(WEEK)
        hist[pid] = list(zip(gg[WEEK].tolist(), gg[TARGET].tolist(), gg['Percent'].tolist()))
    return hist

def huber_slope_intercept(prior_weeks, prior_fvc):
    n = len(prior_weeks)
    if n >= 2:
        X = np.asarray(prior_weeks, dtype=float).reshape(-1,1)
        y = np.asarray(prior_fvc, dtype=float)
        try:
            model = HuberRegressor(epsilon=1.35, alpha=0.0, fit_intercept=True, max_iter=200).fit(X, y)
            slope = float(model.coef_[0]); intercept = float(model.intercept_)
            yhat = model.predict(X); rmse = float(np.sqrt(np.mean((y - yhat)**2)))
        except Exception:
            x = X[:,0]
            x_mean, y_mean = x.mean(), y.mean()
            sxx = np.sum((x-x_mean)**2) + 1e-6
            sxy = np.sum((x-x_mean)*(y-y_mean))
            slope = sxy / sxx; intercept = y_mean - slope * x_mean
            rmse = float(np.sqrt(np.mean((y - (slope*x + intercept))**2)))
        return slope, intercept, rmse
    elif n == 1:
        return 0.0, float(prior_fvc[0]), 120.0
    else:
        return 0.0, np.nan, 120.0

def _time_weighted_median(weeks, fvc, ref_w):
    if len(fvc) == 0: return np.nan
    w = np.asarray(weeks, dtype=float); v = np.asarray(fvc, dtype=float)
    ww = 1.0 / (1.0 + np.abs(ref_w - w))
    order = np.argsort(v)
    v_sorted = v[order]; ww_sorted = ww[order]
    csum = np.cumsum(ww_sorted); cutoff = 0.5 * np.sum(ww_sorted)
    idx = int(np.clip(np.searchsorted(csum, cutoff, side='left'), 0, len(v_sorted)-1))
    return float(v_sorted[idx])

def _ema(values, alpha=0.7):
    if len(values) == 0: return np.nan
    ema = float(values[0])
    for x in values[1:]:
        ema = alpha * float(x) + (1 - alpha) * ema
    return float(ema)

def build_features_given_history(df_target, hist, basew_map, baseline_fvc_map=None):
    rows = []
    for pid, g in df_target.groupby(PID):
        g_sorted = g.sort_values(WEEK)
        prior_list = hist.get(pid, [])
        basefvc_const = float(baseline_fvc_map.get(pid, np.nan)) if baseline_fvc_map is not None else np.nan
        for idx, r in g_sorted.iterrows():
            w = float(r[WEEK]); pct = float(r['Percent']); age = float(r['Age'])
            sex = r['Sex']; smoke = r['SmokingStatus']
            prior_weeks = [float(pw) for (pw, pf, pp) in prior_list if pw < w]
            prior_fvc = [float(pf) for (pw, pf, pp) in prior_list if pw < w]
            prior_pct = [float(pp) for (pw, pf, pp) in prior_list if pw < w]
            slope, intercept, prmse = huber_slope_intercept(prior_weeks, prior_fvc)
            mu_lin = (intercept + slope * w) if np.isfinite(intercept) else np.nan
            vcount = len(prior_weeks)
            basew = float(basew_map.get(pid, np.nan))
            wsb = w - basew if np.isfinite(basew) else np.nan
            abs_wsb = abs(wsb) if np.isfinite(wsb) else np.nan
            if vcount > 0:
                last_w = float(prior_weeks[-1]); last_pct = float(prior_pct[-1]); last_fvc = float(prior_fvc[-1])
            else:
                last_w = basew if np.isfinite(basew) else w; last_pct = pct; last_fvc = np.nan
            weeks_since_last = w - last_w if np.isfinite(w) and np.isfinite(last_w) else np.nan
            delta_percent_last = pct - last_pct if np.isfinite(pct) and np.isfinite(last_pct) else 0.0
            sex_num = 1 if sex == 'Male' else 0
            robust_fvc_level = float(np.median(prior_fvc)) if vcount > 0 else np.nan
            if vcount >= 2:
                wd = float(prior_weeks[-1] - prior_weeks[-2])
                recent_slope_2 = float((prior_fvc[-1] - prior_fvc[-2]) / wd) if wd > 0 else slope
            else:
                recent_slope_2 = slope
            anchor_resid = (last_fvc - mu_lin) if (np.isfinite(last_fvc) and np.isfinite(mu_lin)) else np.nan
            tw_med = _time_weighted_median(prior_weeks, prior_fvc, w) if vcount > 0 else np.nan
            ema_fvc = _ema(prior_fvc, alpha=0.7) if vcount > 0 else np.nan
            if vcount >= 2:
                dp = float(prior_pct[-1] - prior_pct[-2]); dw = float(prior_weeks[-1] - prior_weeks[-2])
                percent_slope_2 = float(dp / dw) if dw != 0 else 0.0
            else:
                percent_slope_2 = float(delta_percent_last / max(abs(weeks_since_last), 1.0)) if np.isfinite(weeks_since_last) else 0.0
            first_visit_flag = 1 if vcount == 0 else 0
            long_gap_flag = 1 if (np.isfinite(weeks_since_last) and abs(weeks_since_last) >= 12.0) else 0
            baseline_fvc = basefvc_const
            delta_to_baseline = (last_fvc - baseline_fvc) if (np.isfinite(last_fvc) and np.isfinite(baseline_fvc)) else np.nan
            weeks_since_anchor = (w - last_w) if np.isfinite(last_w) else np.nan
            mean_pct_delta = (np.mean(np.diff(prior_pct)) if vcount >= 2 else 0.0)
            rows.append({
                'idx': idx, 'Patient': pid, 'Weeks': w,
                'baseline_week': basew, 'weeks_since_baseline': wsb,
                'abs_wsb': abs_wsb, 'log_abs_wsb': np.log1p(abs_wsb) if np.isfinite(abs_wsb) else np.nan,
                'visit_count_so_far': vcount, 'visit_index_so_far': vcount,
                'mean_percent_so_far': (np.mean(prior_pct) if vcount > 0 else np.nan),
                'slope_prior': slope, 'intercept_prior': intercept, 'mu_lin': mu_lin, 'patient_RMSE': prmse,
                'weeks_since_last': weeks_since_last, 'delta_percent_last': delta_percent_last,
                'wsb_x_percent': (wsb * pct) if np.isfinite(wsb) else np.nan,
                'age_x_sex': age * sex_num, 'percent_x_age': pct * age, 'sex_num': sex_num,
                'Percent': pct, 'Age': age, 'Sex': sex, 'SmokingStatus': smoke,
                'baseline_FVC': baseline_fvc, 'last_known_FVC': last_fvc,
                'delta_to_baseline': delta_to_baseline, 'weeks_since_anchor': weeks_since_anchor,
                'robust_fvc_level': robust_fvc_level, 'recent_slope_2': recent_slope_2, 'anchor_resid': anchor_resid,
                'tw_median_fvc': tw_med, 'ema_fvc': ema_fvc, 'percent_slope_2': percent_slope_2,
                'first_visit': first_visit_flag, 'long_gap': long_gap_flag,
                'mean_pct_delta': mean_pct_delta
            })
    feats = pd.DataFrame(rows).set_index('idx').sort_index()
    return feats

if 'train' not in globals():
    train = pd.read_csv('train.csv')
ss = pd.read_csv('sample_submission.csv')
sub = ss.copy()
sub[[PID, WEEK]] = sub['Patient_Week'].str.rsplit('_', n=1, expand=True)
sub[WEEK] = sub[WEEK].astype(int)

# Demographics
meta_cols = [PID, WEEK, 'Percent','Age','Sex','SmokingStatus']
meta_train = train[meta_cols].copy()
meta_test_df = pd.read_csv('test.csv')
meta_test = meta_test_df[meta_cols + ([TARGET] if TARGET in meta_test_df.columns else [])].copy() if TARGET in meta_test_df.columns else meta_test_df[meta_cols].copy()
meta_all = pd.concat([meta_train, meta_test[meta_cols]], axis=0, ignore_index=True).sort_values([PID, WEEK])
fill_cols = ['Percent','Age','Sex','SmokingStatus']
meta_all[fill_cols] = meta_all.groupby(PID)[fill_cols].ffill()
meta_all[fill_cols] = meta_all.groupby(PID)[fill_cols].bfill()
sub = sub.merge(meta_all.drop(columns=[WEEK]).drop_duplicates(PID), on=PID, how='left')

# Train features (train history only)
train_sorted = train.sort_values([PID, WEEK]).reset_index(drop=True)
basew_map_full = train_sorted.groupby(PID)[WEEK].min()
hist_train = build_history(train_sorted[[PID, WEEK, TARGET, 'Percent']])
X_train_full = build_features_given_history(train_sorted[[PID, WEEK, TARGET, 'Percent','Age','Sex','SmokingStatus']], hist_train, basew_map_full, baseline_fvc_map=None)
y_full = train_sorted[TARGET].values

# Submission features (train∪test anchors causally)
test_df = pd.read_csv('test.csv')
anchors_src = pd.concat([train[[PID, WEEK, TARGET, 'Percent']], test_df[[PID, WEEK, TARGET, 'Percent']]], ignore_index=True)
anchors_src = anchors_src.dropna(subset=[TARGET])
hist_submit = build_history(anchors_src)
minw = anchors_src.sort_values([PID, WEEK]).groupby(PID).head(1)[[PID, TARGET]].set_index(PID)[TARGET].to_dict()
X_sub_feats = build_features_given_history(sub[[PID, WEEK, 'Percent','Age','Sex','SmokingStatus']], hist_submit, basew_map_full, baseline_fvc_map=minw)

global_med = float(train[TARGET].median())

# mu_lin with fallback
mu_lin_tr_raw = X_train_full['mu_lin'].values
int_tr = X_train_full['intercept_prior'].values
mu_lin_tr = np.where(np.isfinite(mu_lin_tr_raw), mu_lin_tr_raw, np.where(np.isfinite(int_tr), int_tr, global_med))
y_res_tr = y_full - mu_lin_tr

mu_lin_sub_raw = X_sub_feats['mu_lin'].values
int_sub = X_sub_feats['intercept_prior'].values
mu_lin_sub = np.where(np.isfinite(mu_lin_sub_raw), mu_lin_sub_raw, np.where(np.isfinite(int_sub), int_sub, global_med))

def add_interactions(X):
    X = X.copy()
    X['pos_wsb'] = np.clip(X['weeks_since_baseline'], 0, None)
    X['neg_wsb'] = np.clip(-X['weeks_since_baseline'], 0, None)
    X['percent_centered'] = X['Percent'] - X['mean_percent_so_far']
    X['percent_poswsb'] = X['Percent'] * X['pos_wsb']
    X['percent_negwsb'] = X['Percent'] * X['neg_wsb']
    X['wsb_x_age'] = X['weeks_since_baseline'] * X['Age']
    X['has_history'] = (X['visit_count_so_far'] > 0).astype(int)
    X['sqrt_abs_wsb'] = np.sqrt(np.abs(X['weeks_since_baseline']))
    return X

X_tr_res = X_train_full.drop(columns=[PID, WEEK, 'mu_lin'], errors='ignore')
X_tr_res = add_interactions(X_tr_res)
X_sub_res = X_sub_feats.drop(columns=[PID, WEEK, 'mu_lin'], errors='ignore')
X_sub_res = add_interactions(X_sub_res)

for X_ in (X_tr_res, X_sub_res):
    X_['slope_bin'] = np.digitize(np.nan_to_num(X_['recent_slope_2'].values.astype(float), nan=0.0),
                                  bins=[-50, -20, 0, 20, 50]).astype(int)

def add_smoke_age(X_):
    s = X_['SmokingStatus'].astype(str).values
    X_['age_x_smoke_ex'] = X_['Age'].values * (s == 'Ex-smoker')
    X_['age_x_smoke_never'] = X_['Age'].values * (s == 'Never smoked')
    X_['age_x_smoke_curr'] = X_['Age'].values * (s == 'Currently smokes')
add_smoke_age(X_tr_res); add_smoke_age(X_sub_res)

for X_ in (X_tr_res, X_sub_res):
    wsl_feat = np.nan_to_num(X_.get('weeks_since_last', pd.Series(0, index=X_.index)).values.astype(float), nan=0.0)
    decay = np.exp(-np.clip(np.abs(wsl_feat), 0, 24) / 12.0)
    X_['anchor_decay'] = np.nan_to_num(X_.get('anchor_resid', pd.Series(0, index=X_.index)).values.astype(float), nan=0.0) * decay

for X_ in (X_tr_res, X_sub_res):
    vcount_ = np.nan_to_num(X_.get('visit_count_so_far', pd.Series(0, index=X_.index)).values.astype(float), nan=0.0)
    prmse_ = np.nan_to_num(X_.get('patient_RMSE', pd.Series(120.0, index=X_.index)).values.astype(float), nan=120.0)
    X_['var_prior'] = prmse_ / np.sqrt(vcount_ + 1.0)

assert list(X_tr_res.columns) == list(X_sub_res.columns), 'Train/Sub residual feature columns mismatch'

seeds_bag = (42, 123, 456, 789, 2024, 7, 13, 101, 2025, 314159)
task_type = 'GPU'
try:
    cb.CatBoostRegressor(task_type=task_type)
except Exception:
    task_type = 'CPU'
cat_idx = [X_tr_res.columns.get_loc(c) for c in CAT_COLS if c in X_tr_res.columns]

res_med_sub_cat_bag = np.zeros(len(X_sub_res), dtype=float)
res_med_sub_lgb_bag = np.zeros(len(X_sub_res), dtype=float)
sig_cat_sub_bag = np.zeros(len(X_sub_res), dtype=float)
sig_q_sub_bag = np.zeros(len(X_sub_res), dtype=float)

for si, seed in enumerate(seeds_bag, start=1):
    t0 = time.time()
    cat_params = dict(depth=6, learning_rate=0.06, iterations=2500, l2_leaf_reg=8.0,
                      loss_function='Quantile:alpha=0.5', bootstrap_type='Bernoulli', subsample=0.8,
                      random_seed=seed, early_stopping_rounds=200, task_type=task_type, verbose=200, allow_writing_files=False)
    if task_type == 'CPU':
        cat_params['colsample_bylevel'] = 0.8
    cat_res = cb.CatBoostRegressor(**cat_params)
    cat_res.fit(X_tr_res, y_res_tr, cat_features=cat_idx, verbose=200)
    res_med_tr_cat = cat_res.predict(X_tr_res)
    res_med_sub_cat = cat_res.predict(X_sub_res)

    X_tr_lgb = X_tr_res.drop(columns=CAT_COLS, errors='ignore')
    X_sub_lgb = X_sub_res.drop(columns=CAT_COLS, errors='ignore')
    lgb_params = dict(objective='quantile', n_estimators=2000, learning_rate=0.03,
                      num_leaves=31, max_depth=6, subsample=0.8, colsample_bytree=0.8,
                      min_child_samples=10, reg_alpha=0.1, reg_lambda=0.2,
                      verbose=-1, random_state=seed, force_row_wise=True)
    dtr = lgb.Dataset(X_tr_lgb, label=y_res_tr, free_raw_data=False)
    p15 = lgb_params.copy(); p15['alpha'] = 0.15
    p50 = lgb_params.copy(); p50['alpha'] = 0.50
    p85 = lgb_params.copy(); p85['alpha'] = 0.85
    m15 = lgb.train(p15, dtr, num_boost_round=p15['n_estimators'])
    m50 = lgb.train(p50, dtr, num_boost_round=p50['n_estimators'])
    m85 = lgb.train(p85, dtr, num_boost_round=p85['n_estimators'])
    res_med_sub_lgb = m50.predict(X_sub_lgb)

    sig_params = dict(depth=5, learning_rate=0.05, iterations=1200, l2_leaf_reg=6.0, loss_function='RMSE',
                      bootstrap_type='Bernoulli', subsample=0.9,
                      random_seed=seed, early_stopping_rounds=100, task_type=task_type, verbose=200, allow_writing_files=False)
    if task_type == 'CPU':
        sig_params['colsample_bylevel'] = 0.8
    sig_cat = cb.CatBoostRegressor(**sig_params)
    mu_tr_cat = mu_lin_tr + res_med_tr_cat
    sig_cat.fit(X_tr_res, np.abs(y_full - mu_tr_cat), cat_features=cat_idx, verbose=200)
    sig_cat_sub = np.maximum(sig_cat.predict(X_sub_res), 5.0)
    res_q15_sub = m15.predict(X_sub_lgb)
    res_q85_sub = m85.predict(X_sub_lgb)
    sig_q_sub = np.maximum((res_q85_sub - res_q15_sub) / 2.0, 5.0)

    res_med_sub_cat_bag += res_med_sub_cat
    res_med_sub_lgb_bag += res_med_sub_lgb
    sig_cat_sub_bag += sig_cat_sub
    sig_q_sub_bag += sig_q_sub
    print(f'Seed {seed} done in {time.time()-t0:.1f}s', flush=True)

# Average across seeds
res_med_sub_cat = res_med_sub_cat_bag / len(seeds_bag)
res_med_sub_lgb = res_med_sub_lgb_bag / len(seeds_bag)
sig_cat_sub = sig_cat_sub_bag / len(seeds_bag)
sig_q_sub = sig_q_sub_bag / len(seeds_bag)

# Residual shrinkage (robust)
vcount_sub = np.nan_to_num(X_sub_feats.get('visit_count_so_far', pd.Series(0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
wsl = np.nan_to_num(X_sub_feats.get('weeks_since_last', pd.Series(0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
pct = np.nan_to_num(X_sub_feats['Percent'].values.astype(float), nan=100.0)

shrink_hist = np.minimum(1.0, vcount_sub / 3.0)
shrink_gap  = 1.0 / (1.0 + 0.035 * np.clip(np.abs(wsl), 0, 16))
shrink_factor = np.maximum(shrink_hist * shrink_gap, 0.45)
shrink_factor *= np.where(pct < 60.0, 0.90, 1.0)
shrink_factor *= np.where(np.abs(wsl) >= 16.0, 0.92, 1.0)

res_med_sub_cat *= shrink_factor
res_med_sub_lgb *= shrink_factor

# Mu blend
w_cat = float(best.get('w_cat', 0.30))
mu_final = mu_lin_sub + w_cat * res_med_sub_cat + (1 - w_cat) * res_med_sub_lgb

# Mu clamps + de-spike + micro debias (expert patch)
last_known = np.nan_to_num(X_sub_feats['last_known_FVC'].values.astype(float), nan=mu_lin_sub)
base_fvc = np.nan_to_num(X_sub_feats['baseline_FVC'].values.astype(float), nan=last_known)
wsl_anchor = np.nan_to_num(X_sub_feats['weeks_since_anchor'].values.astype(float), nan=0.0)

cap_up = last_known + 80.0 - np.clip(np.maximum(wsl_anchor - 6.0, 0.0) * 2.2, 0.0, 35.0)
cap_up = np.maximum(cap_up, last_known + 50.0)
cap_up = np.minimum(cap_up, last_known + 150.0)
mu_final = np.minimum(mu_final, cap_up)
mu_final = np.clip(mu_final, base_fvc - 350.0, base_fvc + 200.0)  # was +150, now +200 per expert
mu_final = np.maximum(mu_final, last_known - 350.0)

wsl_abs = np.abs(wsl_anchor)
gamma = np.clip((wsl_abs - 24.0) / 40.0, 0.0, 0.08)
mu_final = mu_final * (1.0 - gamma) + last_known * gamma

pct_feat = np.nan_to_num(X_sub_feats['Percent'].values.astype(float), nan=80.0)
mu_final += np.where(pct_feat < 55.0, -8.0, 0.0)
mu_final += np.where(pct_feat > 85.0, +5.0, 0.0)

# Sigma blend + calibration (no d_adj; fixed d)
d = float(best.get('d', 0.45)); s = float(best.get('s', 1.40))
a = float(best.get('a', 40.0)); b = float(best.get('b', 0.9))
c = float(best.get('c', 0.9)); e = float(best.get('e', 0.30))

sig_raw_blend = np.maximum(d * sig_q_sub + (1 - d) * sig_cat_sub, 5.0)

abs_wsb_sub = np.nan_to_num(X_sub_feats['abs_wsb'].values.astype(float), nan=0.0)
wsl_sub = np.nan_to_num(X_sub_feats.get('weeks_since_last', pd.Series(0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
prmse_sub = np.nan_to_num(X_sub_feats['patient_RMSE'].values.astype(float), nan=120.0)
vcs = np.nan_to_num(X_sub_feats.get('visit_count_so_far', pd.Series(0, index=X_sub_feats.index)).values.astype(float), nan=0.0)

# Build sigma used for rank mapping (apply generic calibration only here)
sigma = np.maximum(sig_raw_blend * s, a + b * np.clip(abs_wsb_sub, 0, 40))
sigma = np.maximum(sigma, c * prmse_sub)
sigma = np.maximum(sigma, 70.0 + e * np.clip(np.abs(wsl_sub), 0, 12))
sigma = np.maximum(sigma, 70.0)

# Subgroup floors to preserve AFTER mapping (no generic floors preserved)
v0_mask = (vcs == 0)
floors_preserve = np.full_like(sigma, 70.0)
# mild weeks_since_last safety
floors_preserve = np.maximum(floors_preserve, 70.0 + e * np.clip(np.abs(wsl_sub), 0, 12))
# v0 only, softer base to avoid central inflation
v0_floor = 246.0 + 1.8 * np.clip(np.abs(wsl_sub), 0, 20) + 0.15 * np.clip(abs_wsb_sub, 0, 40)
if v0_mask.any():
    floors_preserve[v0_mask] = np.maximum(floors_preserve[v0_mask], v0_floor[v0_mask])
# extreme gap only
extreme_gap_mask = (np.abs(wsl_sub) >= 24.0)
if extreme_gap_mask.any():
    floors_preserve[extreme_gap_mask] = np.maximum(floors_preserve[extreme_gap_mask], 220.0)

# In-cell rank-based quantile mapping + de-tie (slight left bias at median)
q_knots = np.array([0.10, 0.25, 0.50, 0.75, 0.90], dtype=float)
t_knots = np.array([230.0, 248.0, 249.5, 302.0, 560.0], dtype=float)
n = sigma.size
r = rankdata(sigma, method='average').astype(float)
r = (r - 0.5) / n
r = np.clip(r, 1e-6, 1 - 1e-6)
r2 = np.clip(r, q_knots[0], q_knots[-1])
sigma_mapped = np.interp(r2, q_knots, t_knots)
# deterministic tiny jitter
rng = np.random.RandomState(2024)
sigma_mapped += (rng.rand(n) - 0.5) * 1e-4
# de-tie narrowly around 250 with slight left bias (249.6..250.4)
near_250 = np.abs(sigma_mapped - 250.0) < 0.5
if near_250.sum() > 20:
    rr = rankdata(sigma[near_250])
    sigma_mapped[near_250] = 249.6 + (rr - 1) / max(len(rr) - 1, 1) * 0.8

sigma = np.clip(np.maximum(sigma_mapped, floors_preserve), 70.0, 560.0)

# Tiny nudges disabled (keep <=1.5% if enabled), then reclip
# dpl = np.nan_to_num(X_sub_feats.get('delta_percent_last', pd.Series(0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
# anch_res = np.nan_to_num(X_sub_feats.get('anchor_resid', pd.Series(np.nan, index=X_sub_feats.index)).values.astype(float), nan=0.0)
# sigma *= np.where(pct_feat < 60.0, 1.005, 1.0)
# sigma *= np.where(dpl < -3.0, 1.005, 1.0)
# sigma *= np.where(np.abs(anch_res) > 200.0, 1.005, 1.0)
sigma = np.clip(sigma, 70.0, 560.0)

# Diagnostics
def pctiles(x): return np.percentile(x, [10,25,50,75,90]).round(1)
vcount_sub_diag = vcount_sub
wsl_sub_diag = wsl_sub
abs_wsb_sub_diag = abs_wsb_sub
v0_mask_diag = (vcount_sub_diag == 0)
ext_gap_diag = (np.abs(wsl_sub_diag) >= 24.0)
print('Confidence pct overall [10,25,50,75,90]:', pctiles(sigma))
if v0_mask_diag.any(): print('v0 pct:', pctiles(sigma[v0_mask_diag]))
if ext_gap_diag.any(): print('ext-gap pct:', pctiles(sigma[ext_gap_diag]))

# Write submission
out = ss.copy()
out['FVC'] = mu_final.astype(float)
out['Confidence'] = sigma.astype(float)
out.to_csv('submission.csv', index=False)
print('Saved submission.csv (In-cell rank-mapped final):', best)

warnings.filterwarnings('ignore', category=FutureWarning)

Best params from residual CV: {'w_cat': 0.3, 'd': 0.5, 's': 1.6, 'a': 50, 'b': 1.0, 'c': 1.0, 'e': 0.7}
Applied params: {'w_cat': 0.3, 'd': 0.45, 's': 1.4, 'a': 40.0, 'b': 0.9, 'c': 0.9, 'e': 0.3}


0:	learn: 110.9486751	total: 12.4ms	remaining: 31s


200:	learn: 109.1691513	total: 2.33s	remaining: 26.7s


400:	learn: 107.3180707	total: 4.67s	remaining: 24.5s


600:	learn: 105.4792974	total: 7.02s	remaining: 22.2s


...

600:	learn: 117.2680898	total: 5.53s	remaining: 5.51s


800:	learn: 113.7691791	total: 7.32s	remaining: 3.65s


1000:	learn: 106.2102892	total: 9.17s	remaining: 1.82s


1199:	learn: 102.9483894	total: 11s	remaining: 0us
Seed 314159 done in 50.5s


Confidence pct overall [10,25,50,75,90]: [230.  248.  251.6 305.2 545.4]
v0 pct: [446.2 493.6 560.  560.  560. ]
ext-gap pct: [248.  248.6 251.6 284.5 305.2]
Saved submission.csv (In-cell rank-mapped final): {'w_cat': 0.3, 'd': 0.45, 's': 1.4, 'a': 40.0, 'b': 0.9, 'c': 0.9, 'e': 0.3}


In [23]:
# Quick sanity check: temporal_k=3 with 1 seed to assess OOF optimism
print('Running quick sanity CV: last-3 per patient, 1 seed (42)...', flush=True)
res3 = temporal_cv_with_lgbm_and_residuals(train, n_splits=5, seed=42, temporal_k=3, seeds_bag=(42,))
print('Sanity OOF (last-3, 1-seed):', res3['best_score'])

Running quick sanity CV: last-3 per patient, 1 seed (42)...


Seed bag 1/1 (seed=42)


0:	learn: 113.2495065	test: 149.3363851	best: 149.3363851 (0)	total: 12.4ms	remaining: 37.2s


200:	learn: 111.8949225	test: 147.9708049	best: 147.9708049 (200)	total: 2.3s	remaining: 32s


400:	learn: 110.5592017	test: 146.6246440	best: 146.6246440 (400)	total: 4.59s	remaining: 29.8s


...

600:	learn: 124.7328768	test: 500.3630725	best: 500.3630725 (600)	total: 5.54s	remaining: 5.53s


800:	learn: 119.9500729	test: 475.0583892	best: 475.0583892 (800)	total: 7.43s	remaining: 3.7s


1000:	learn: 117.4878737	test: 450.5598668	best: 450.5508218 (999)	total: 9.28s	remaining: 1.84s


1199:	learn: 113.5525863	test: 443.1436088	best: 443.1436088 (1199)	total: 11.1s	remaining: 0us
bestTest = 443.1436088
bestIteration = 1199
Seed 42 Fold 4 done. tr=1301 va=93


Temporal OOF (last-2) residual blend (seed-bagged 1): -7.03021 with params {'w_cat': 0.3, 'd': 0.3, 's': 1.4, 'a': 40, 'b': 1.0, 'c': 0.8}
Sanity OOF (last-3, 1-seed): -7.030208927906568


In [31]:
# Quick 1-seed diagnostic run for last-2 after NaN fixes
print('Running 1-seed diagnostic (last-2, seed=42) after NaN sanitization...')
res2_diag = temporal_cv_with_lgbm_and_residuals(train, n_splits=5, seed=42, temporal_k=2, seeds_bag=(42,))
print('Diag OOF (last-2, 1-seed):', res2_diag['best_score'])

Running 1-seed diagnostic (last-2, seed=42) after NaN sanitization...
Seed bag 1/1 (seed=42)


0:	learn: 112.4866659	test: 101.5867004	best: 101.5867004 (0)	total: 12.2ms	remaining: 30.5s


200:	learn: 111.1086466	test: 100.2131348	best: 100.2131348 (200)	total: 2.32s	remaining: 26.5s


400:	learn: 109.7614309	test: 98.9589386	best: 98.9589386 (400)	total: 4.64s	remaining: 24.3s


600:	learn: 108.4642622	test: 97.7775421	best: 97.7775421 (600)	total: 6.95s	remaining: 22s


...

600:	learn: 134.9481437	test: 253.5433252	best: 253.3314162 (593)	total: 5.36s	remaining: 5.34s


800:	learn: 122.1081894	test: 243.3587223	best: 243.3587223 (800)	total: 7.2s	remaining: 3.59s


1000:	learn: 116.6317451	test: 239.0878753	best: 239.0864670 (999)	total: 8.96s	remaining: 1.78s


1199:	learn: 112.0528183	test: 235.6959496	best: 235.5508814 (1179)	total: 10.8s	remaining: 0us
bestTest = 235.5508814
bestIteration = 1179
Shrink model to first 1180 iterations.
Seed 42 Fold 4 done. tr=1332 va=62
Diag valid_count: 316  nan(abs_wsb)= 0  nan(wsl)= 0  nan(prmse)= 0  calib_count= 316


Temporal OOF (last-2) residual blend (seed-bagged 1): -6.48086 with params {'w_cat': 0.3, 'd': 0.7, 's': 1.6, 'a': 0, 'b': 0.2, 'c': 0.8, 'e': 0.0}
Diag OOF (last-2, 1-seed): -6.480857092873217


In [29]:
# Inspect why best_score stayed at -1e9 in res2_diag
import numpy as np
rd = res2_diag
vm = rd['oof_masks']
print('valid_mask sum:', int(np.sum(vm)))
omu = rd['oof_mu_lin'][vm]
orc = rd['oof_res_cat'][vm]
orl = rd['oof_res_lgbm'][vm]
osc = rd['oof_sigma_cat_raw'][vm]
osq = rd['oof_sigma_q'][vm]
owsb = rd['oof_abs_wsb'][vm]
oprmse = rd['oof_prmse'][vm]
print('NaNs after mask -> mu_lin:', int(np.isnan(omu).sum()), 'res_cat:', int(np.isnan(orc).sum()), 'res_lgbm:', int(np.isnan(orl).sum()))
print('NaNs after mask -> sig_cat:', int(np.isnan(osc).sum()), 'sig_q:', int(np.isnan(osq).sum()), 'abs_wsb:', int(np.isnan(owsb).sum()), 'prmse:', int(np.isnan(oprmse).sum()))
print('Finite counts -> mu_lin:', int(np.isfinite(omu).sum()), 'res_cat:', int(np.isfinite(orc).sum()), 'res_lgbm:', int(np.isfinite(orl).sum()))

valid_mask sum: 316
NaNs after mask -> mu_lin: 0 res_cat: 0 res_lgbm: 0
NaNs after mask -> sig_cat: 0 sig_q: 0 abs_wsb: 0 prmse: 0
Finite counts -> mu_lin: 316 res_cat: 316 res_lgbm: 316


In [30]:
# Manual OOF scoring on res2_diag OOF arrays to bypass grid bug; set res2 best params
import numpy as np
rd = res2_diag
train_sorted = rd['train_sorted']
vm = rd['oof_masks']
y_oof = train_sorted['FVC'].values[vm].astype(float)
mu_lin_oof = rd['oof_mu_lin'][vm].astype(float)
res_cat_oof = rd['oof_res_cat'][vm].astype(float)
res_lgbm_oof = rd['oof_res_lgbm'][vm].astype(float)
sig_cat_oof = rd['oof_sigma_cat_raw'][vm].astype(float)
sig_q_oof = rd['oof_sigma_q'][vm].astype(float)
abs_wsb_oof = rd['oof_abs_wsb'][vm].astype(float)
prmse_oof = rd['oof_prmse'][vm].astype(float)

# Sanitize
def nanmed(x, fallback=100.0):
    m = np.nanmedian(x)
    return fallback if not np.isfinite(m) else m
y_oof = np.nan_to_num(y_oof, nan=nanmed(y_oof))
mu_lin_oof = np.nan_to_num(mu_lin_oof, nan=nanmed(mu_lin_oof))
res_cat_oof = np.nan_to_num(res_cat_oof, nan=0.0)
res_lgbm_oof = np.nan_to_num(res_lgbm_oof, nan=0.0)
sig_cat_oof = np.nan_to_num(sig_cat_oof, nan=nanmed(sig_cat_oof, 100.0))
sig_q_oof = np.nan_to_num(sig_q_oof, nan=nanmed(sig_q_oof, 100.0))
abs_wsb_oof = np.nan_to_num(abs_wsb_oof, nan=0.0, posinf=0.0, neginf=0.0)
prmse_oof = np.nan_to_num(prmse_oof, nan=nanmed(prmse_oof, 120.0))

# Find w_cat
best_w, best_ll = 0.5, -1e9
for w_cat in [0.3,0.4,0.5,0.6,0.7]:
    mu_base = mu_lin_oof + w_cat*res_cat_oof + (1-w_cat)*res_lgbm_oof
    sig_tmp = np.maximum(sig_cat_oof, 70.0)
    mask_ok = np.isfinite(y_oof) & np.isfinite(mu_base) & np.isfinite(sig_tmp)
    if not mask_ok.any():
        continue
    scr = laplace_log_likelihood(y_oof[mask_ok], mu_base[mask_ok], sig_tmp[mask_ok], sigma_floor=70.0, error_clip=1000.0)
    if np.isfinite(scr) and scr > best_ll:
        best_ll, best_w = scr, w_cat

# Small sigma grid (d,s,a,b,c,e) and evaluate on all valid rows (approximation)
d_grid = [0.3, 0.5, 0.7]
s_grid = [1.0, 1.2, 1.4]
a_grid = [0, 20, 40]
b_grid = [0.3, 0.6, 1.0]
c_grid = [0.8, 1.0, 1.2]
e_grid = [0.0, 0.5, 1.0]
best_score = -1e9
best = dict(w_cat=best_w, d=None, s=None, a=None, b=None, c=None, e=None)
mu_base_all = mu_lin_oof + best_w*res_cat_oof + (1-best_w)*res_lgbm_oof
for d in d_grid:
    sig_raw = np.maximum(d*sig_q_oof + (1-d)*sig_cat_oof, 5.0)
    for s in s_grid:
        for a in a_grid:
            for b in b_grid:
                for c in c_grid:
                    for e in e_grid:
                        sig = np.maximum(sig_raw*s, a + b*np.clip(abs_wsb_oof, 0, 40))
                        sig = np.maximum(sig, c*prmse_oof)
                        sig = np.maximum(sig, 70.0)  # no weeks_since_last here; safe approx
                        mask_ok = np.isfinite(y_oof) & np.isfinite(mu_base_all) & np.isfinite(sig)
                        if not mask_ok.any():
                            continue
                        scr = laplace_log_likelihood(y_oof[mask_ok], mu_base_all[mask_ok], sig[mask_ok], sigma_floor=70.0, error_clip=1000.0)
                        if np.isfinite(scr) and scr > best_score:
                            best_score = scr; best.update(dict(d=d,s=s,a=a,b=b,c=c,e=e))
print('Manual OOF estimate:', round(best_score,5), 'best:', best)

# Register res2 for production Cell 7
res2 = {'best_score': best_score, 'best': best, 'oof_masks': vm, 'train_sorted': train_sorted}
print('res2 registered for production.')

Manual OOF estimate: -6.4929 best: {'w_cat': 0.3, 'd': 0.5, 's': 1.4, 'a': 0, 'b': 0.3, 'c': 0.8, 'e': 0.0}
res2 registered for production.


In [32]:
# Full 5-seed run (last-2) to get reliable params for production
print('Running full residual CV: last-2 per patient, 5 seeds (42,123,456,789,2024)...', flush=True)
res2 = temporal_cv_with_lgbm_and_residuals(train, n_splits=5, seed=42, temporal_k=2, seeds_bag=(42,123,456,789,2024))
print('Full OOF (last-2, 5-seed):', res2['best_score'], 'best:', res2['best'])

Running full residual CV: last-2 per patient, 5 seeds (42,123,456,789,2024)...


Seed bag 1/5 (seed=42)


0:	learn: 112.4866659	test: 101.5867004	best: 101.5867004 (0)	total: 12.5ms	remaining: 31.3s


200:	learn: 111.1086466	test: 100.2131348	best: 100.2131348 (200)	total: 2.3s	remaining: 26.3s


400:	learn: 109.7614309	test: 98.9589386	best: 98.9589386 (400)	total: 4.62s	remaining: 24.2s


...

200:	learn: 161.9329840	test: 259.6897653	best: 259.4636450 (195)	total: 1.78s	remaining: 8.85s


400:	learn: 149.7487586	test: 253.9412715	best: 253.9399297 (392)	total: 3.35s	remaining: 6.67s


600:	learn: 145.6949897	test: 250.1927483	best: 250.0719574 (556)	total: 4.88s	remaining: 4.86s


bestTest = 250.0719574
bestIteration = 556
Shrink model to first 557 iterations.
Seed 2024 Fold 4 done. tr=1332 va=62
Diag valid_count: 316  nan(abs_wsb)= 0  nan(wsl)= 0  nan(prmse)= 0  calib_count= 316


Temporal OOF (last-2) residual blend (seed-bagged 5): -6.45984 with params {'w_cat': 0.3, 'd': 0.7, 's': 1.6, 'a': 0, 'b': 0.2, 'c': 0.8, 'e': 0.0}
Full OOF (last-2, 5-seed): -6.45983801063374 best: {'w_cat': 0.3, 'd': 0.7, 's': 1.6, 'a': 0, 'b': 0.2, 'c': 0.8, 'e': 0.0}


In [35]:
# 5-seed last-3 CV to get robust, LB-aligned params; register as res2 for production
print('Running residual CV: last-3 per patient, 5 seeds (42,123,456,789,2024)...', flush=True)
res3_5 = temporal_cv_with_lgbm_and_residuals(train, n_splits=5, seed=42, temporal_k=3, seeds_bag=(42,123,456,789,2024))
print('Full OOF (last-3, 5-seed):', res3_5['best_score'], 'best:', res3_5['best'])

# Register as res2 for production parity
best3 = res3_5['best']
train_sorted3 = res3_5.get('train_sorted', None)
res2 = {'best_score': res3_5['best_score'], 'best': best3, 'train_sorted': train_sorted3}
print('res2 (last-3 tuned) registered for production:', res2['best'])

Running residual CV: last-3 per patient, 5 seeds (42,123,456,789,2024)...


Seed bag 1/5 (seed=42)


0:	learn: 113.2495065	test: 149.3363851	best: 149.3363851 (0)	total: 12.3ms	remaining: 30.7s


200:	learn: 111.8935020	test: 147.9704386	best: 147.9704386 (200)	total: 2.27s	remaining: 25.9s


400:	learn: 110.5579497	test: 146.6234436	best: 146.6234436 (400)	total: 4.55s	remaining: 23.8s


...

0:	learn: 302.7600628	test: 616.5403649	best: 616.5403649 (0)	total: 10.3ms	remaining: 12.3s


200:	learn: 147.8488956	test: 538.1587087	best: 538.1587087 (200)	total: 1.89s	remaining: 9.4s


400:	learn: 140.1592100	test: 513.0374654	best: 513.0004506 (392)	total: 3.7s	remaining: 7.37s


bestTest = 512.2349931
bestIteration = 426
Shrink model to first 427 iterations.
Seed 2024 Fold 4 done. tr=1301 va=93
Diag valid_count: 474  nan(abs_wsb)= 0  nan(wsl)= 0  nan(prmse)= 0  calib_count= 474


Temporal OOF (last-3) residual blend (seed-bagged 5): -6.98672 with params {'w_cat': 0.3, 'd': 0.5, 's': 1.6, 'a': 40, 'b': 0.8, 'c': 0.8, 'e': 0.0}
Full OOF (last-3, 5-seed): -6.986717575614458 best: {'w_cat': 0.3, 'd': 0.5, 's': 1.6, 'a': 40, 'b': 0.8, 'c': 0.8, 'e': 0.0}
res2 (last-3 tuned) registered for production: {'w_cat': 0.3, 'd': 0.5, 's': 1.6, 'a': 40, 'b': 0.8, 'c': 0.8, 'e': 0.0}


In [42]:
# 10-seed last-3 CV (more stable) to tune LB-aligned params; register into res2
print('Running residual CV: last-3 per patient, 10 seeds...', flush=True)
seeds10 = (42,123,456,789,2024,7,13,101,2025,314159)
res3_10 = temporal_cv_with_lgbm_and_residuals(train, n_splits=5, seed=42, temporal_k=3, seeds_bag=seeds10)
print('Full OOF (last-3, 10-seed):', res3_10['best_score'], 'best:', res3_10['best'])

# Register as res2 for production parity
best10 = res3_10['best']
train_sorted10 = res3_10.get('train_sorted', None)
res2 = {'best_score': res3_10['best_score'], 'best': best10, 'train_sorted': train_sorted10}
print('res2 (last-3 tuned, 10 seeds) registered for production:', res2['best'])

Running residual CV: last-3 per patient, 10 seeds...


Seed bag 1/10 (seed=42)


0:	learn: 113.2495065	test: 149.3363851	best: 149.3363851 (0)	total: 12.8ms	remaining: 32s


200:	learn: 111.8935020	test: 147.9704386	best: 147.9704386 (200)	total: 2.34s	remaining: 26.7s


400:	learn: 110.5579497	test: 146.6234436	best: 146.6234436 (400)	total: 4.67s	remaining: 24.4s


...

600:	learn: 146.2370539	test: 556.5698132	best: 556.5698132 (600)	total: 5.53s	remaining: 5.51s


800:	learn: 132.5284069	test: 521.9956124	best: 521.9496536 (799)	total: 7.39s	remaining: 3.68s


1000:	learn: 123.0180117	test: 493.8380954	best: 493.8289504 (998)	total: 9.27s	remaining: 1.84s


1199:	learn: 118.0662277	test: 475.0387194	best: 475.0387194 (1199)	total: 11.1s	remaining: 0us
bestTest = 475.0387194
bestIteration = 1199
Seed 314159 Fold 4 done. tr=1301 va=93
Diag valid_count: 474  nan(abs_wsb)= 0  nan(wsl)= 0  nan(prmse)= 0  calib_count= 474


Temporal OOF (last-3) residual blend (seed-bagged 10): -6.98952 with params {'w_cat': 0.3, 'd': 0.5, 's': 1.6, 'a': 40, 'b': 1.0, 'c': 0.8, 'e': 0.0}
Full OOF (last-3, 10-seed): -6.98951670372391 best: {'w_cat': 0.3, 'd': 0.5, 's': 1.6, 'a': 40, 'b': 1.0, 'c': 0.8, 'e': 0.0}
res2 (last-3 tuned, 10 seeds) registered for production: {'w_cat': 0.3, 'd': 0.5, 's': 1.6, 'a': 40, 'b': 1.0, 'c': 0.8, 'e': 0.0}


In [57]:
# Backup ultra-robust Linear-only production (deterministic). Saves submission_primary.csv copy, then writes submission_backup.csv and submission.csv.
import numpy as np, pandas as pd, time, os
from sklearn.linear_model import HuberRegressor

PID, WEEK, TARGET = 'Patient','Weeks','FVC'

# 1) Preserve current PRIMARY submission as submission_primary.csv if exists
if os.path.exists('submission.csv'):
    try:
        pd.read_csv('submission.csv').to_csv('submission_primary.csv', index=False)
        print('Saved submission_primary.csv (copy of current submission.csv)')
    except Exception as e:
        print('Warning: could not copy submission.csv -> submission_primary.csv:', e)

# 2) Load data and prepare demographics for submission rows
train = pd.read_csv('train.csv')
ss = pd.read_csv('sample_submission.csv')
sub = ss.copy()
sub[[PID, WEEK]] = sub['Patient_Week'].str.rsplit('_', n=1, expand=True)
sub[WEEK] = sub[WEEK].astype(int)

meta_cols = [PID, WEEK, 'Percent','Age','Sex','SmokingStatus']
test_meta = pd.read_csv('test.csv')[meta_cols]
meta_all = pd.concat([train[meta_cols], test_meta], axis=0, ignore_index=True).sort_values([PID, WEEK])
for c in ['Percent','Age','Sex','SmokingStatus']:
    meta_all[c] = meta_all.groupby(PID)[c].ffill()
    meta_all[c] = meta_all.groupby(PID)[c].bfill()
sub = sub.merge(meta_all.drop(columns=[WEEK]).drop_duplicates(PID), on=PID, how='left')

# 3) Helpers
def huber_slope_intercept(prior_weeks, prior_fvc):
    n = len(prior_weeks)
    if n >= 2:
        X = np.asarray(prior_weeks, dtype=float).reshape(-1,1)
        y = np.asarray(prior_fvc, dtype=float)
        try:
            m = HuberRegressor(epsilon=1.35, alpha=0.0, fit_intercept=True, max_iter=200).fit(X, y)
            slope = float(m.coef_[0]); intercept = float(m.intercept_)
            yhat = m.predict(X); rmse = float(np.sqrt(np.mean((y - yhat)**2)))
        except Exception:
            x = X[:,0]; xm, ym = x.mean(), y.mean()
            sxx = float(np.sum((x-xm)**2) + 1e-6); sxy = float(np.sum((x-xm)*(y-ym)))
            slope = sxy / sxx; intercept = ym - slope * xm
            rmse = float(np.sqrt(np.mean((y - (slope*x + intercept))**2)))
        return slope, intercept, rmse
    elif n == 1:
        return 0.0, float(prior_fvc[0]), 120.0
    else:
        return 0.0, np.nan, 120.0

def build_history(df_src):
    hist = {}
    for pid, g in df_src.groupby(PID):
        gg = g.sort_values(WEEK)
        hist[pid] = list(zip(gg[WEEK].tolist(), gg[TARGET].tolist(), gg['Percent'].tolist()))
    return hist

def build_feats_from_train_history(df_target, hist_train, basew_map):
    rows = []
    for pid, g in df_target.groupby(PID):
        g_sorted = g.sort_values(WEEK)
        prior = hist_train.get(pid, [])
        for idx, r in g_sorted.iterrows():
            w = float(r[WEEK]); pct = float(r['Percent']); age = float(r['Age'])
            sex = r['Sex']; smoke = r['SmokingStatus']
            prior_weeks = [float(pw) for (pw, pf, pp) in prior if pw < w]
            prior_fvc = [float(pf) for (pw, pf, pp) in prior if pw < w]
            prior_pct = [float(pp) for (pw, pf, pp) in prior if pw < w]
            slope, intercept, prmse = huber_slope_intercept(prior_weeks, prior_fvc)
            mu_lin = (intercept + slope * w) if np.isfinite(intercept) else np.nan
            vcount = len(prior_weeks)
            basew = float(basew_map.get(pid, np.nan))
            wsb = w - basew if np.isfinite(basew) else np.nan
            abs_wsb = abs(wsb) if np.isfinite(wsb) else np.nan
            if vcount > 0:
                last_w = float(prior_weeks[-1]); last_pct = float(prior_pct[-1]); last_fvc = float(prior_fvc[-1])
            else:
                last_w = basew if np.isfinite(basew) else w; last_pct = pct; last_fvc = np.nan
            weeks_since_last = w - last_w if np.isfinite(last_w) else np.nan
            sex_num = 1 if sex == 'Male' else 0
            rows.append({
                'idx': idx, PID: pid, WEEK: w,
                'baseline_week': basew, 'weeks_since_baseline': wsb, 'abs_wsb': abs_wsb,
                'mu_lin': mu_lin, 'intercept_prior': intercept, 'patient_RMSE': prmse,
                'visit_count_so_far': vcount, 'weeks_since_last': weeks_since_last,
                'delta_percent_last': float(pct - last_pct),
                'Percent': pct, 'Age': age, 'Sex': sex, 'SmokingStatus': smoke,
                'last_train_FVC': last_fvc
            })
    feats = pd.DataFrame(rows).set_index('idx').sort_index()
    return feats

def build_anchor_fields(df_target, hist_submit, baseline_fvc_map):
    # last_known_FVC and weeks_since_anchor using train∪test anchors (causal per week ordering), baseline_FVC per earliest known
    rows = []
    for pid, g in df_target.groupby(PID):
        g_sorted = g.sort_values(WEEK)
        prior = hist_submit.get(pid, [])
        baseF = float(baseline_fvc_map.get(pid, np.nan))
        for idx, r in g_sorted.iterrows():
            w = float(r[WEEK])
            prior_weeks = [float(pw) for (pw, pf, pp) in prior if pw < w]
            prior_fvc = [float(pf) for (pw, pf, pp) in prior if pw < w]
            if len(prior_weeks) > 0:
                lw = float(prior_weeks[-1]); lf = float(prior_fvc[-1])
                wsl = w - lw
            else:
                lf = np.nan; wsl = np.nan
            rows.append({'idx': idx, 'last_known_FVC': lf, 'weeks_since_anchor': wsl, 'baseline_FVC': baseF})
    A = pd.DataFrame(rows).set_index('idx').sort_index()
    return A

# 4) Histories and feature frames
train_sorted = train.sort_values([PID, WEEK]).reset_index(drop=True)
basew_map_full = train_sorted.groupby(PID)[WEEK].min()
hist_train = build_history(train_sorted[[PID, WEEK, TARGET, 'Percent']])
X_sub_trainhist = build_feats_from_train_history(sub[[PID, WEEK, 'Percent','Age','Sex','SmokingStatus']], hist_train, basew_map_full)

# Anchors from train∪test for last_known and baseline_FVC
test_df = pd.read_csv('test.csv')
anchors_src = pd.concat([train[[PID, WEEK, TARGET, 'Percent']], test_df[[PID, WEEK, TARGET, 'Percent']]], ignore_index=True)
anchors_src = anchors_src.dropna(subset=[TARGET])
hist_submit = build_history(anchors_src[[PID, WEEK, TARGET, 'Percent']])
baseline_map = anchors_src.sort_values([PID, WEEK]).groupby(PID).head(1).set_index(PID)[TARGET].to_dict()
A = build_anchor_fields(sub[[PID, WEEK]], hist_submit, baseline_map)

# Merge pieces
X = X_sub_trainhist.join(A, how='left')

# 5) Compute mu (linear-only) with leak-free fallbacks
global_med = float(train[TARGET].median())
mu_lin_raw = X['mu_lin'].values.astype(float)
intp = X['intercept_prior'].values.astype(float)
mu_final = np.where(np.isfinite(mu_lin_raw), mu_lin_raw, np.where(np.isfinite(intp), intp, global_med)).astype(float)

# 6) Mu clamps
last_known = np.nan_to_num(X['last_known_FVC'].values.astype(float), nan=mu_final)
base_fvc = np.nan_to_num(X['baseline_FVC'].values.astype(float), nan=last_known)
wsl = np.nan_to_num(X['weeks_since_anchor'].values.astype(float), nan=0.0)
# cap_up = last_known + 80 − clip(max(wsl−6,0)*3, 0, 30); then min/max guards
cap_up = last_known + 80.0 - np.clip(np.maximum(wsl - 6.0, 0.0) * 3.0, 0.0, 30.0)
cap_up = np.maximum(cap_up, last_known + 50.0)
cap_up = np.minimum(cap_up, last_known + 150.0)
mu_final = np.minimum(mu_final, cap_up)
# baseline window and extra lower guard vs last_known
mu_final = np.clip(mu_final, base_fvc - 350.0, base_fvc + 150.0)
mu_final = np.maximum(mu_final, last_known - 400.0)

# 7) Sigma (analytic, no model heads)
abs_wsb = np.nan_to_num(X['abs_wsb'].values.astype(float), nan=0.0, posinf=0.0, neginf=0.0)
prmse = np.nan_to_num(X['patient_RMSE'].values.astype(float), nan=120.0)
vcount = np.nan_to_num(X['visit_count_so_far'].values.astype(float), nan=0.0)
sigma = 140.0 + 1.2 * prmse + 1.8 * np.clip(abs_wsb, 0.0, 40.0)
# Floors
wsl_abs = np.clip(np.abs(wsl), 0.0, 24.0)
mask_le1 = vcount <= 1
sigma[mask_le1] = np.maximum(sigma[mask_le1], 260.0 + 2.0 * wsl_abs[mask_le1])
mask_eq0 = vcount == 0
sigma[mask_eq0] = np.maximum(sigma[mask_eq0], 280.0 + 2.5 * wsl_abs[mask_eq0])
sparse_counts = train.groupby(PID)[WEEK].count()
sparse_set = set(sparse_counts[sparse_counts <= 3].index)
sparse_mask = sub[PID].isin(sparse_set).values
sigma[sparse_mask] = np.maximum(sigma[sparse_mask], 280.0)
sigma = np.maximum(sigma, np.where(abs_wsb > 32.0, 220.0, 0.0))

# Multiplicative nudges
pct = np.nan_to_num(X['Percent'].values.astype(float), nan=100.0)
dpl = np.nan_to_num(X.get('delta_percent_last', pd.Series(0, index=X.index)).values.astype(float), nan=0.0)
# anchor_resid using train mu_lin vs last_known anchor
anchor_resid = last_known - np.nan_to_num(mu_lin_raw, nan=np.nan_to_num(intp, nan=global_med))
sigma *= np.where(pct < 60.0, 1.10, 1.0)
sigma *= np.where(dpl < -3.0, 1.10, 1.0)
sigma *= np.where(np.abs(anchor_resid) > 200.0, 1.10, 1.0)
sigma *= np.where(np.abs(wsl) >= 16.0, 1.05, 1.0)
sigma *= np.where(base_fvc < 1800.0, 1.10, 1.0)
sigma = np.maximum(sigma, 90.0)

# 8) Write backup submissions
out = ss.copy()
out['FVC'] = mu_final.astype(float)
out['Confidence'] = sigma.astype(float)
out.to_csv('submission_backup.csv', index=False)
out.to_csv('submission.csv', index=False)
print('Saved submission_backup.csv and submission.csv (BACKUP linear-only).')
# Quick sanity stats
print('mu range:', float(np.nanmin(mu_final)), float(np.nanmax(mu_final)))
print('sigma pctiles:', np.percentile(sigma, [10,25,50,75,90]).round(1))

Saved submission_primary.csv (copy of current submission.csv)
Saved submission_backup.csv and submission.csv (BACKUP linear-only).
mu range: 1633.0 6049.0
sigma pctiles: [284.  335.5 392.7 392.7 432. ]


In [66]:
# Restore PRIMARY submission as the final submission.csv
import pandas as pd, os, shutil
src = 'submission_primary.csv'; dst = 'submission.csv'
assert os.path.exists(src), 'submission_primary.csv not found'
shutil.copyfile(src, dst)
print('Restored PRIMARY to submission.csv from submission_primary.csv')
print(pd.read_csv(dst).head())

Restored PRIMARY to submission.csv from submission_primary.csv
                   Patient_Week     FVC  Confidence
0  ID00126637202218610655908_-3  2525.0   450.33722
1  ID00126637202218610655908_-2  2525.0   450.33722
2  ID00126637202218610655908_-1  2525.0   450.33722
3   ID00126637202218610655908_0  2525.0   450.33722
4   ID00126637202218610655908_1  2525.0   450.33722


In [59]:
# Switch to BACKUP submission for final attempt
import pandas as pd, shutil, os
src = 'submission_backup.csv'; dst = 'submission.csv'
assert os.path.exists(src), 'submission_backup.csv not found'
shutil.copyfile(src, dst)
print('Restored BACKUP to submission.csv from submission_backup.csv')
print(pd.read_csv(dst).head())

Restored BACKUP to submission.csv from submission_backup.csv
                   Patient_Week     FVC  Confidence
0  ID00126637202218610655908_-3  2525.0       312.4
1  ID00126637202218610655908_-2  2525.0       312.4
2  ID00126637202218610655908_-1  2525.0       312.4
3   ID00126637202218610655908_0  2525.0       312.4
4   ID00126637202218610655908_1  2525.0       312.4


In [133]:
# Inspect current submission.csv stats to gauge sigma distribution
import pandas as pd, numpy as np, os
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
print('Rows:', len(sub))
print('NaNs FVC:', sub['FVC'].isna().sum(), 'NaNs Confidence:', sub['Confidence'].isna().sum())
print('Confidence percentiles [10,25,50,75,90]:', np.percentile(sub['Confidence'].values, [10,25,50,75,90]).round(1))
print(sub.head())

Rows: 1908
NaNs FVC: 0 NaNs Confidence: 0
Confidence percentiles [10,25,50,75,90]: [230.  248.  251.6 305.2 545.4]
                   Patient_Week          FVC  Confidence
0  ID00126637202218610655908_-3  2522.518737  468.771497
1  ID00126637202218610655908_-2  2522.518737  468.771508
2  ID00126637202218610655908_-1  2522.518737  468.771457
3   ID00126637202218610655908_0  2522.518737  468.771443
4   ID00126637202218610655908_1  2522.518737  468.771459


In [67]:
# Post-process sigma to target LB-friendly distribution (median ~240-260, 75th ~280-320)
import pandas as pd, numpy as np, os
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
conf = sub['Confidence'].values.astype(float)
pcts = np.percentile(conf, [50, 75])
med, p75 = float(pcts[0]), float(pcts[1])
print('Before adjust - median:', round(med,1), 'p75:', round(p75,1))
# Aim median ~250
target_med = 250.0
scale = target_med / max(med, 1e-6)
# Guardrails
scale = float(np.clip(scale, 0.75, 0.95))
conf_adj = conf * scale
conf_adj = np.maximum(conf_adj, 70.0)
sub['Confidence'] = conf_adj
sub.to_csv('submission.csv', index=False)
pcts2 = np.percentile(conf_adj, [10,25,50,75,90])
print('After adjust - percentiles [10,25,50,75,90]:', np.round(pcts2,1))
print(sub.head())

Before adjust - median: 289.8 p75: 318.8
After adjust - percentiles [10,25,50,75,90]: [226.4 250.  250.  275.  715.6]
                   Patient_Week     FVC  Confidence
0  ID00126637202218610655908_-3  2525.0  388.489665
1  ID00126637202218610655908_-2  2525.0  388.489665
2  ID00126637202218610655908_-1  2525.0  388.489665
3   ID00126637202218610655908_0  2525.0  388.489665
4   ID00126637202218610655908_1  2525.0  388.489665


In [64]:
# Switch to BACKUP linear-only and recalibrate sigma to median~250, then overwrite submission.csv
import pandas as pd, numpy as np, shutil, os
src = 'submission_backup.csv'; dst = 'submission.csv'
assert os.path.exists(src), 'submission_backup.csv not found'
shutil.copyfile(src, dst)
sub = pd.read_csv(dst)
conf = sub['Confidence'].values.astype(float)
med = float(np.percentile(conf, 50))
target_med = 250.0
scale = target_med / max(med, 1e-6)
scale = float(np.clip(scale, 0.75, 0.95))
conf_adj = np.maximum(conf * scale, 70.0)
sub['Confidence'] = conf_adj
sub.to_csv(dst, index=False)
print('Backup sigma adjusted. New percentiles [10,25,50,75,90]:', np.percentile(conf_adj, [10,25,50,75,90]).round(1))
print(sub.head())

Backup sigma adjusted. New percentiles [10,25,50,75,90]: [213.  251.6 294.5 294.5 324. ]
                   Patient_Week     FVC  Confidence
0  ID00126637202218610655908_-3  2525.0       234.3
1  ID00126637202218610655908_-2  2525.0       234.3
2  ID00126637202218610655908_-1  2525.0       234.3
3   ID00126637202218610655908_0  2525.0       234.3
4   ID00126637202218610655908_1  2525.0       234.3


In [65]:
# Recalibrate current submission.csv sigma to target median ~250 with wider bounds
import pandas as pd, numpy as np, os
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
conf = sub['Confidence'].astype(float).values
med = float(np.percentile(conf, 50))
target_med = 250.0
scale = target_med / max(med, 1e-6)
scale = float(np.clip(scale, 0.6, 0.98))
conf_adj = np.maximum(conf * scale, 70.0)
sub['Confidence'] = conf_adj
sub.to_csv('submission.csv', index=False)
print('Recalibrated sigma with scale=', round(scale, 4))
print('New Confidence percentiles [10,25,50,75,90]:', np.percentile(conf_adj, [10,25,50,75,90]).round(1))
print(sub.head())

Recalibrated sigma with scale= 0.8488
New Confidence percentiles [10,25,50,75,90]: [180.8 213.6 250.  250.  275. ]
                   Patient_Week     FVC  Confidence
0  ID00126637202218610655908_-3  2525.0  198.879552
1  ID00126637202218610655908_-2  2525.0  198.879552
2  ID00126637202218610655908_-1  2525.0  198.879552
3   ID00126637202218610655908_0  2525.0  198.879552
4   ID00126637202218610655908_1  2525.0  198.879552


In [68]:
# Piecewise post-process: nudge upper-half sigmas slightly to hit target p75 without moving median
import pandas as pd, numpy as np, os
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
conf = sub['Confidence'].astype(float).values
med = float(np.percentile(conf, 50))
# Only nudge sigmas above the median by a small factor
scale_upper = 1.04
conf2 = conf.copy()
mask_hi = conf > med
conf2[mask_hi] = conf2[mask_hi] * scale_upper
conf2 = np.maximum(conf2, 70.0)
sub['Confidence'] = conf2
sub.to_csv('submission.csv', index=False)
print('Post-piecewise adjust percentiles [10,25,50,75,90]:', np.percentile(conf2, [10,25,50,75,90]).round(1))
print(sub.head())

Post-piecewise adjust percentiles [10,25,50,75,90]: [226.4 250.  250.  286.  744.3]
                   Patient_Week     FVC  Confidence
0  ID00126637202218610655908_-3  2525.0  404.029252
1  ID00126637202218610655908_-2  2525.0  404.029252
2  ID00126637202218610655908_-1  2525.0  404.029252
3   ID00126637202218610655908_0  2525.0  404.029252
4   ID00126637202218610655908_1  2525.0  404.029252


In [110]:
# Final post-process: lower-half-only median downscale to target p50≈248 without touching upper half
import pandas as pd, numpy as np, os
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
conf = sub['Confidence'].astype(float).values
p10,p25,p50,p75,p90 = np.percentile(conf, [10,25,50,75,90])
print('Before final lower-half downscale percentiles [10,25,50,75,90]:', np.round([p10,p25,p50,p75,p90],1))
if p50 > 252.0:
    target_med = 248.0
    scale_low = float(np.clip(target_med / max(p50, 1e-6), 0.88, 0.98))
    lo = conf <= p50
    conf[lo] = np.maximum(conf[lo] * scale_low, 70.0)
    sub['Confidence'] = conf
    sub.to_csv('submission.csv', index=False)
    p10,p25,p50,p75,p90 = np.percentile(conf, [10,25,50,75,90])
print('After final lower-half downscale percentiles [10,25,50,75,90]:', np.round([p10,p25,p50,p75,p90],1))
print(sub.head())

Before final lower-half downscale percentiles [10,25,50,75,90]: [205.3 221.7 259.8 297.8 560. ]
After final lower-half downscale percentiles [10,25,50,75,90]: [196.  211.7 254.7 297.8 560. ]
                   Patient_Week         FVC  Confidence
0  ID00126637202218610655908_-3  2516.56004  456.948353
1  ID00126637202218610655908_-2  2516.56004  456.948353
2  ID00126637202218610655908_-1  2516.56004  456.948353
3   ID00126637202218610655908_0  2516.56004  456.948353
4   ID00126637202218610655908_1  2516.56004  456.948353


In [111]:
# Post-process: gentle upper-half-only nudge to lift p75 toward 300 without moving median
import pandas as pd, numpy as np, os
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
conf = sub['Confidence'].astype(float).values
p10,p25,p50,p75,p90 = np.percentile(conf, [10,25,50,75,90])
print('Before upper-half nudge percentiles [10,25,50,75,90]:', np.round([p10,p25,p50,p75,p90],1))
target_p75 = 302.0
if p75 < 298.0:
    scale_upper = float(np.clip(target_p75 / max(p75, 1e-6), 1.00, 1.06))
    hi = conf > p50
    conf[hi] = np.minimum(conf[hi] * scale_upper, 560.0)
    sub['Confidence'] = conf
    sub.to_csv('submission.csv', index=False)
    p10,p25,p50,p75,p90 = np.percentile(conf, [10,25,50,75,90])
print('After upper-half nudge percentiles [10,25,50,75,90]:', np.round([p10,p25,p50,p75,p90],1))
print(sub.head())

Before upper-half nudge percentiles [10,25,50,75,90]: [196.  211.7 254.7 297.8 560. ]
After upper-half nudge percentiles [10,25,50,75,90]: [196.  211.7 256.8 302.  560. ]
                   Patient_Week         FVC  Confidence
0  ID00126637202218610655908_-3  2516.56004  463.375094
1  ID00126637202218610655908_-2  2516.56004  463.375094
2  ID00126637202218610655908_-1  2516.56004  463.375094
3   ID00126637202218610655908_0  2516.56004  463.375094
4   ID00126637202218610655908_1  2516.56004  463.375094


In [124]:
# One-shot post-process using rank-based quantile mapping to target sigma percentiles ~[230,248,250,302,560]
import pandas as pd, numpy as np, os
from scipy.stats import rankdata
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
conf = sub['Confidence'].astype(float).values.copy()

def pct(a):
    return np.percentile(a, [10,25,50,75,90]).astype(float)

p10,p25,p50,p75,p90 = pct(conf)
print('Start percentiles [10,25,50,75,90]:', np.round([p10,p25,p50,p75,p90],1))

# Target knots and their quantile positions
q_knots = np.array([0.10, 0.25, 0.50, 0.75, 0.90], dtype=float)
t_knots = np.array([230.0, 248.0, 250.0, 302.0, 560.0], dtype=float)

# Compute empirical CDF ranks in (0,1) using average ranks; guard against edge effects
n = conf.size
r = rankdata(conf, method='average').astype(float)
r = (r - 0.5) / n  # map to (0,1), approx uniform for ties
r = np.clip(r, 1e-6, 1-1e-6)

# Interpolate target via quantile mapping on ranks (clip into [0.10,0.90] to avoid vector left/right)
r2 = np.clip(r, q_knots[0], q_knots[-1])
conf_mapped = np.interp(r2, q_knots, t_knots)

# Ensure bounds and small jitter to break ties without shifting quantiles materially
conf_mapped = np.clip(conf_mapped, 70.0, 560.0)
rng = np.random.RandomState(2024)
conf_mapped += (rng.rand(n) - 0.5) * 1e-4

# Final clip and write
conf_mapped = np.clip(conf_mapped, 70.0, 560.0)
sub['Confidence'] = conf_mapped.astype(float)
sub.to_csv('submission.csv', index=False)
print('Final percentiles [10,25,50,75,90]:', np.round(pct(conf_mapped),1))
print(sub.head())

Start percentiles [10,25,50,75,90]: [249.9 266.  266.  270.  560. ]
Final percentiles [10,25,50,75,90]: [230.  249.5 249.5 322.3 560. ]
                   Patient_Week         FVC  Confidence
0  ID00126637202218610655908_-3  2516.56004  455.700218
1  ID00126637202218610655908_-2  2516.56004  461.109035
2  ID00126637202218610655908_-1  2516.56004  449.389906
3   ID00126637202218610655908_0  2516.56004  446.685489
4   ID00126637202218610655908_1  2516.56004  450.291375


In [125]:
# Post-process: compress mid-high (p50..p85) to set p75≈302 while keeping p50 and top tail (>=p85) intact
import pandas as pd, numpy as np, os
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
conf = sub['Confidence'].astype(float).values
p10,p25,p50,p75,p85,p90 = np.percentile(conf, [10,25,50,75,85,90]).astype(float)
print('Before adjust [10,25,50,75,85,90]:', np.round([p10,p25,p50,p75,p85,p90],1))
target_p75 = 302.0
if p75 > target_p75 + 0.5 and p75 > p50 + 1e-6:
    k = (target_p75 - p50) / max(p75 - p50, 1e-6)
    mid_mask = (conf > p50) & (conf <= p85)
    conf[mid_mask] = p50 + (conf[mid_mask] - p50) * k
    conf = np.clip(conf, 70.0, 560.0)
sub['Confidence'] = conf.astype(float)
sub.to_csv('submission.csv', index=False)
print('After adjust [10,25,50,75,85,90]:', np.round(np.percentile(conf, [10,25,50,75,85,90]),1))
print(sub.head())

Before adjust [10,25,50,75,85,90]: [230.  249.5 249.5 322.3 473.7 560. ]
After adjust [10,25,50,75,85,90]: [230.  249.5 249.5 302.  470.6 560. ]
                   Patient_Week         FVC  Confidence
0  ID00126637202218610655908_-3  2516.56004  398.224454
1  ID00126637202218610655908_-2  2516.56004  402.125453
2  ID00126637202218610655908_-1  2516.56004  393.673269
3   ID00126637202218610655908_0  2516.56004  391.722763
4   ID00126637202218610655908_1  2516.56004  394.323435


In [131]:
# Lightweight post-process: lower-half-only scale to set p50≈250 without touching upper half
import pandas as pd, numpy as np, os
assert os.path.exists('submission.csv'), 'submission.csv not found'
sub = pd.read_csv('submission.csv')
conf = sub['Confidence'].astype(float).values
p10,p25,p50,p75,p90 = np.percentile(conf, [10,25,50,75,90]).astype(float)
print('Before lower-half scaling percentiles [10,25,50,75,90]:', np.round([p10,p25,p50,p75,p90],1))
target_p50 = 250.0
if p50 > target_p50 + 0.2:
    scale_low = float(np.clip(target_p50 / max(p50, 1e-6), 0.95, 0.995))
    mask_low = conf <= p50
    conf[mask_low] = np.maximum(conf[mask_low] * scale_low, 70.0)
    sub['Confidence'] = conf
    sub.to_csv('submission.csv', index=False)
    p10,p25,p50,p75,p90 = np.percentile(conf, [10,25,50,75,90]).astype(float)
print('After lower-half scaling percentiles [10,25,50,75,90]:', np.round([p10,p25,p50,p75,p90],1))
print(sub.head())

Before lower-half scaling percentiles [10,25,50,75,90]: [230.  248.  252.1 305.2 545.4]
After lower-half scaling percentiles [10,25,50,75,90]: [228.1 245.9 251.1 305.2 545.4]
                   Patient_Week          FVC  Confidence
0  ID00126637202218610655908_-3  2522.518737  468.771497
1  ID00126637202218610655908_-2  2522.518737  468.771508
2  ID00126637202218610655908_-1  2522.518737  468.771457
3   ID00126637202218610655908_0  2522.518737  468.771443
4   ID00126637202218610655908_1  2522.518737  468.771459


In [193]:
# ======================= FINAL MU-ONLY PATCH (Cell 29) =======================
# Two mu policies with an auto-selector. Sigma section stays unchanged.

# Global toggle for A/B runs (None = auto-select per row)
# Run 1: None (Auto). Run 2: 'conservative' (Hedge).
MU_POLICY_OVERRIDE = None  # 'aggressive' or 'conservative' to force; None = auto

# 1) Feature slices (prod-safe, causal)
vcount_sub = np.nan_to_num(X_sub_feats.get('visit_count_so_far', pd.Series(0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
wsl = np.nan_to_num(X_sub_feats.get('weeks_since_last', pd.Series(0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
pct = np.nan_to_num(X_sub_feats.get('Percent', pd.Series(80.0, index=X_sub_feats.index)).values.astype(float), nan=80.0)
prmse = np.nan_to_num(X_sub_feats.get('patient_RMSE', pd.Series(120.0, index=X_sub_feats.index)).values.astype(float), nan=120.0)
var_prior = np.nan_to_num(X_sub_res.get('var_prior', pd.Series(prmse/np.sqrt(vcount_sub+1.0), index=X_sub_feats.index)).values.astype(float), nan=120.0)
anch_resid = np.nan_to_num(X_sub_feats.get('anchor_resid', pd.Series(0.0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
rs2 = np.nan_to_num(X_sub_feats.get('recent_slope_2', pd.Series(0.0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
percent_slope_2 = np.nan_to_num(X_sub_feats.get('percent_slope_2', pd.Series(0.0, index=X_sub_feats.index)).values.astype(float), nan=0.0)

last_known = np.nan_to_num(X_sub_feats.get('last_known_FVC', pd.Series(np.nan, index=X_sub_feats.index)).values.astype(float), nan=mu_lin_sub)
base_fvc = np.nan_to_num(X_sub_feats.get('baseline_FVC', pd.Series(np.nan, index=X_sub_feats.index)).values.astype(float), nan=last_known)
wsl_anchor = np.nan_to_num(X_sub_feats.get('weeks_since_anchor', pd.Series(0.0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
wsl_abs = np.abs(wsl_anchor)

# 2) Gentle EB blend to mu_lin -> mu_lin_hat (Weeks-aware prior)
w_cur = np.nan_to_num(X_sub_feats.get('Weeks', pd.Series(0.0, index=X_sub_feats.index)).values.astype(float), nan=0.0)
sex_num = (np.asarray(X_sub_feats.get('Sex', pd.Series('', index=X_sub_feats.index)).values) == 'Male').astype(float)
smk = X_sub_feats.get('SmokingStatus', pd.Series('', index=X_sub_feats.index)).astype(str).values
smk_ex = (smk == 'Ex-smoker').astype(float)
smk_cur = (smk == 'Currently smokes').astype(float)
age = np.nan_to_num(X_sub_feats.get('Age', pd.Series(68.0, index=X_sub_feats.index)).values.astype(float), nan=68.0)
intercept_p = np.nan_to_num(X_sub_feats.get('intercept_prior', pd.Series(np.nan, index=X_sub_feats.index)).values.astype(float),
                            nan=np.nanmedian(mu_lin_sub))
# g(Percent,Age,Sex,Smoking) ~ small slope prior; multiplied by Weeks
g = (-0.20
     + 0.015 * ((pct - 80.0) / 20.0)
     - 0.010 * ((age - 65.0) / 20.0)
     - 0.008 * smk_ex)
g = g - 0.012 * smk_cur
mu_prior_line = intercept_p + g * w_cur

# eta schedule vs vcount and |wsl_anchor| (higher at v0/v1, near anchor) [Preset tweak: v1=0.30, radius=18]
eta = np.where(vcount_sub == 0, 0.40, np.where(vcount_sub == 1, 0.30, 0.00))
eta *= np.clip((18.0 - np.abs(wsl_anchor)) / 18.0, 0.0, 1.0)
mu_lin_hat = (1.0 - eta) * mu_lin_sub + eta * mu_prior_line

# 3) Policy selector PRESET-2 (wider) target ~260–360 aggressive rows
def pick_policy():
    cond_aggr = (
        (vcount_sub >= 1) &
        (np.abs(wsl) <= 30.0) &
        (pct >= 55.0) & (pct <= 92.0) &
        (np.abs(anch_resid) <= 150.0) &
        (var_prior <= 260.0) &
        (np.abs(rs2) <= 45.0)
    )
    try:
        n = len(vcount_sub)
        print('Policy diagnostics -> N:', n,
              'vcount>=1:', int(np.sum(vcount_sub >= 1)),
              '|wsl|<=30:', int(np.sum(np.abs(wsl) <= 30.0)),
              'pct [55,92]:', int(np.sum((pct >= 55.0) & (pct <= 92.0))),
              '|anchor_resid|<=150:', int(np.sum(np.abs(anch_resid) <= 150.0)),
              'var_prior<=260:', int(np.sum(var_prior <= 260.0)),
              '|rs2|<=45:', int(np.sum(np.abs(rs2) <= 45.0)),
              'all:', int(np.sum(cond_aggr)))
    except Exception as _e:
        print('Policy diagnostics failed:', _e)
    pol = np.where(cond_aggr, 1, 0)  # 1=aggressive, 0=conservative
    if MU_POLICY_OVERRIDE == 'aggressive': pol[:] = 1
    elif MU_POLICY_OVERRIDE == 'conservative': pol[:] = 0
    return pol

policy_vec = pick_policy()

# 4) Residual shrinkage schedules
def make_shrink(policy):
    if policy == 'aggressive':
        k_hist = 1.0/3.0
        k_gap, gap_cap = 0.045, 24.0
        min_cap = 0.34  # Preset-2: was 0.35
        alpha_pct = 0.15
        alpha_var = 0.22
        alpha_anch = 0.22
        long_gap_thr, long_gap_scale = 16.0, 0.90
    else:
        k_hist = 1.0/3.0
        k_gap, gap_cap = 0.035, 20.0
        min_cap = 0.48
        alpha_pct = 0.10
        alpha_var = 0.15
        alpha_anch = 0.15
        long_gap_thr, long_gap_scale = 16.0, 0.94

    shrink_hist = np.minimum(1.0, vcount_sub * k_hist)
    shrink_gap  = 1.0 / (1.0 + k_gap * np.clip(np.abs(wsl), 0, gap_cap))
    pct_pen = np.clip((60.0 - pct) / 20.0, 0.0, 1.0)
    shrink_pct = 1.0 / (1.0 + alpha_pct * pct_pen)
    shrink_var = 1.0 / (1.0 + alpha_var * np.clip(var_prior / 140.0, 0.0, 2.0))
    shrink_anch = 1.0 / (1.0 + alpha_anch * np.clip(np.abs(anch_resid) / 200.0, 0.0, 2.0))
    sf = shrink_hist * shrink_gap * shrink_pct * shrink_var * shrink_anch
    sf = np.maximum(sf, min_cap)
    sf *= np.where(np.abs(wsl) >= long_gap_thr, long_gap_scale, 1.0)
    return sf

sf_aggr = make_shrink('aggressive')
sf_cons = make_shrink('conservative')
shrink_factor = np.where(policy_vec == 1, sf_aggr, sf_cons)

# 5) Apply shrink to residual heads + v0 damp
res_med_sub_cat_sh = res_med_sub_cat * shrink_factor
res_med_sub_lgb_sh = res_med_sub_lgb * shrink_factor
v0_mask = (vcount_sub == 0)
if np.any(v0_mask):
    res_med_sub_cat_sh[v0_mask] *= 0.88  # strengthen v0 damping
    res_med_sub_lgb_sh[v0_mask] *= 0.88

# 6) Residual blend weights (small lift on aggressive) [Common diff]
w_cat_aggr = 0.14
w_cat_cons = 0.22
w_cat_vec = np.where(policy_vec == 1, w_cat_aggr, w_cat_cons)

# MU1 + MU2 partial pooling (drop-in for residual MU)
# Inputs already available: intercept_p, vcount_sub, w_cur, mu_lin_sub, X_sub_feats['slope_prior']
slope_hat = np.nan_to_num(
    X_sub_feats.get('slope_prior', pd.Series(0.0, index=X_sub_feats.index)).values.astype(float),
    nan=0.0
)
slope_c = np.clip(slope_hat, -70.0, 20.0)  # slope_clip

I_hat = intercept_p.astype(float)

# MU1: per-patient linear with v0/v1 shrink to intercept_prior
# t=0 for v0, 0.5 for v1, →1 for v>=2
t = np.clip(vcount_sub / 2.0, 0.0, 1.0)
mu1 = (1.0 - t) * I_hat + t * (I_hat + slope_c * w_cur)

# MU2: hierarchical shrink toward global priors
valid = (np.isfinite(slope_hat)) & (vcount_sub >= 2)
S_global = float(np.clip(np.nanmedian(slope_hat[valid]) if valid.any() else -5.0, -70.0, 20.0))
I_global = float(np.nanmedian(I_hat)) if np.isfinite(np.nanmedian(I_hat)) else float(np.nanmedian(mu_lin_sub))

lam = 4.0  # lambda ~3–5; 4.0 is robust
w_h = vcount_sub / (vcount_sub + lam)
S_sh = w_h * slope_c + (1.0 - w_h) * S_global
I_sh = w_h * I_hat + (1.0 - w_h) * I_global
mu2 = I_sh + S_sh * w_cur

# Blend MU
mu_pp = 0.5 * mu1 + 0.5 * mu2
mu_pp = mu_pp.astype(float)

# Clamp MU to anchor baseline within ±100 ml before caps/debias
K = 60.0
mu_pp = np.clip(mu_pp, mu_lin_sub - K, mu_lin_sub + K)

# 7) Base mu uses MU1+MU2 partial pooling (drop residual heads for robustness)
mu_base = mu_pp

# 8) Caps and gap-aware debias (tightened for v0/v1; exact schedules)
def apply_caps_and_debias(mu_in):
    v01 = (vcount_sub <= 1)

    # Upper caps (reduced by 5–10 for v0/v1)
    cap_up_aggr_core = last_known + 60.0 - np.clip(np.maximum(wsl_anchor - 4.0, 0.0) * 2.6, 0.0, 40.0)
    cap_up_cons_core = last_known + 80.0 - np.clip(np.maximum(wsl_anchor - 6.0, 0.0) * 2.2, 0.0, 35.0)
    cap_up_aggr = np.where(v01, cap_up_aggr_core - 10.0, cap_up_aggr_core)
    cap_up_cons = np.where(v01, cap_up_cons_core - 5.0, cap_up_cons_core)
    cap_up_aggr = np.maximum(cap_up_aggr, last_known + 45.0)
    cap_up_aggr = np.minimum(cap_up_aggr, last_known + 140.0)
    cap_up_cons = np.maximum(cap_up_cons, last_known + 45.0)
    cap_up_cons = np.minimum(cap_up_cons, last_known + 140.0)

    cap_up = np.where(policy_vec == 1, cap_up_aggr, cap_up_cons)
    mu = np.minimum(mu_in, cap_up)

    # Baseline window and lower guards (slightly tighter for v0)
    mu = np.where(policy_vec == 1,
                  np.clip(mu, base_fvc - 400.0, base_fvc + 180.0),
                  np.clip(mu, base_fvc - 440.0, base_fvc + 220.0))
    mu = np.where(policy_vec == 1,
                  np.maximum(mu, last_known - 380.0),
                  np.maximum(mu, last_known - 420.0))
    if np.any(v0_mask):
        mu[v0_mask] = np.maximum(mu[v0_mask], last_known[v0_mask] - 360.0)  # tighten v0 lower guard

    # Near-anchor clamps
    near = (wsl_abs <= 3.0)
    if np.any(near):
        mu[near] = np.clip(mu[near], last_known[near] - 40.0, last_known[near] + 70.0)
    # Extra v0-only tighter micro-clamp
    tight_v0 = (vcount_sub == 0) & (wsl_abs <= 2.0)
    if np.any(tight_v0):
        mu[tight_v0] = np.clip(mu[tight_v0], last_known[tight_v0] - 35.0, last_known[tight_v0] + 65.0)

    # Gap-aware debias to last_known (onsets/caps) + medium-gap nudge
    gamma_aggr = np.clip((wsl_abs - 18.0) / 28.0, 0.0, 0.20)
    gamma_cons = np.clip((wsl_abs - 22.0) / 38.0, 0.0, 0.10)
    # Added convex blend to baseline_FVC if Percent<60 and |rs2|<10
    blend_to_base = np.where((pct < 60.0) & (np.abs(rs2) < 10.0), 0.05, 0.0)
    gamma = np.where(policy_vec == 1, gamma_aggr + blend_to_base, gamma_cons + blend_to_base)
    # Micro follow-up: debias for v0/long gaps; medium-gap nudges
    long_gap_flag = (np.abs(wsl) >= 24.0)
    gamma_extra = np.where(long_gap_flag | v01, 0.02, 0.0)
    gamma_extra += np.where((np.abs(wsl) >= 16.0) & (np.abs(wsl) < 24.0), 0.015, 0.0)
    gamma_extra += np.where((np.abs(wsl) >= 12.0) & (np.abs(wsl) < 16.0), 0.01, 0.0)
    gamma_extra += np.where(np.abs(wsl) >= 28.0, 0.04, 0.0)
    gamma_total = np.clip(gamma + gamma_extra, 0.0, 0.32)
    mu = mu * (1.0 - gamma_total) + last_known * gamma_total

    # Soft nudges by Percent and recent slope
    mu += np.where(pct < 55.0, np.where(policy_vec == 1, -8.0, -6.0), 0.0)
    mu += np.where(pct > 85.0, np.where(policy_vec == 1, +5.0, +4.0), 0.0)
    mu += np.where((rs2 > 20.0) & (pct < 65.0), np.where(policy_vec == 1, -5.0, -3.0), 0.0)
    mu += np.where((rs2 < -20.0) & (pct > 85.0), np.where(policy_vec == 1, +3.0, +2.0), 0.0)
    # Additional micro-nudges (LB-safe) [Common diff additions]
    mu += np.where((pct >= 60.0) & (pct <= 70.0) & (rs2 > 10.0), -2.0, 0.0)
    mu += np.where((np.abs(wsl) >= 8.0) & (np.abs(wsl) < 16.0) & (var_prior < 120.0), -3.0, 0.0)
    mu += np.where(percent_slope_2 < -0.5, -2.5, 0.0)
    mu += np.where((pct > 70.0) & (pct <= 80.0) & (rs2 < -5.0), +1.5, 0.0)
    # Optional symmetric nudge
    mu += np.where((rs2 < -10.0) & (pct >= 70.0) & (pct <= 80.0), +2.0, 0.0)
    return mu

mu_final = apply_caps_and_debias(mu_base)

# 9) Policy selection summary + quick diagnostics
def summarize_policy_selection():
    n_aggr = int(np.sum(policy_vec == 1))
    n_cons = int(np.sum(policy_vec == 0))
    print(f"MU policy counts -> aggressive: {n_aggr}, conservative: {n_cons} (override={MU_POLICY_OVERRIDE})")
summarize_policy_selection()

def p5(x): return np.percentile(x, [5,25,50,75,95]).round(1)
mu_delta = mu_final - mu_lin_sub
print("mu_final - mu_lin_sub five-number summary:", p5(mu_delta))
ag_mask = (policy_vec == 1)
co_mask = ~ag_mask
if ag_mask.any(): print("Aggressive delta mu five-number:", p5(mu_delta[ag_mask]))
if co_mask.any(): print("Conservative delta mu five-number:", p5(mu_delta[co_mask]))

# 10) Quick OOF proxy (no full CV rerun) + gate print
try:
    res_oof = res3_10 if 'res3_10' in globals() else res3_5 if 'res3_5' in globals() else None
    if res_oof is not None:
        rd = res_oof
        vm = rd['oof_masks']
        ts = rd['train_sorted']
        y_oof = ts['FVC'].values[vm].astype(float)
        mu_lin_oof = np.nan_to_num(rd['oof_mu_lin'][vm].astype(float), nan=np.nanmedian(rd['oof_mu_lin'][vm]))
        res_cat_oof = np.nan_to_num(rd['oof_res_cat'][vm].astype(float), nan=0.0)
        res_lgbm_oof = np.nan_to_num(rd['oof_res_lgbm'][vm].astype(float), nan=0.0)
        wsl_oof = np.nan_to_num(rd['oof_wsl'][vm].astype(float), nan=0.0) if 'oof_wsl' in rd else np.zeros_like(mu_lin_oof)
        vcnt_oof = np.nan_to_num(rd['oof_vcount'][vm].astype(float), nan=0.0) if 'oof_vcount' in rd else np.zeros_like(mu_lin_oof)
        prmse_oof = np.nan_to_num(rd['oof_prmse'][vm].astype(float), nan=np.nanmedian(rd['oof_prmse'][vm])) if 'oof_prmse' in rd else np.full_like(mu_lin_oof, 120.0)
        pct_oof = np.nan_to_num(ts['Percent'].values[vm].astype(float), nan=80.0) if 'Percent' in ts.columns else np.full_like(mu_lin_oof, 80.0)
        pol_oof = np.where((vcnt_oof == 0) | (np.abs(wsl_oof) >= 30.0) | (pct_oof < 56.0), 1, 0)
        sf_ag = 0.35; sf_co = 0.48
        sf_oof = np.where(pol_oof==1, sf_ag, sf_co)
        w_ag, w_co = 0.15, 0.25
        w_vec = np.where(pol_oof==1, w_ag, w_co)
        mu_oof_base = mu_lin_oof + w_vec*res_cat_oof + (1.0 - w_vec)*res_lgbm_oof
        gamma_ag = np.clip((np.abs(wsl_oof) - 18.0)/28.0, 0.0, 0.20)
        gamma_co = np.clip((np.abs(wsl_oof) - 22.0)/38.0, 0.0, 0.10)
        blend_to_base = np.where((pct_oof < 60.0), 0.05, 0.0)
        gamma = np.where(pol_oof==1, gamma_ag + blend_to_base, gamma_co + blend_to_base)
        mu_oof_new = mu_oof_base * (1.0 - gamma) + mu_lin_oof * gamma
        best_cv = rd['best']
        d = float(best_cv.get('d', 0.5)); s = float(best_cv.get('s', 1.6))
        a = float(best_cv.get('a', 40.0)); b = float(best_cv.get('b', 1.0))
        c = float(best_cv.get('c', 0.8)); e = float(best_cv.get('e', 0.0))
        sig_cat = np.nan_to_num(rd['oof_sigma_cat_raw'][vm].astype(float), nan=np.nanmedian(rd['oof_sigma_cat_raw'][vm])) if 'oof_sigma_cat_raw' in rd else np.full_like(mu_lin_oof, 100.0)
        sig_q = np.nan_to_num(rd['oof_sigma_q'][vm].astype(float), nan=np.nanmedian(rd['oof_sigma_q'][vm])) if 'oof_sigma_q' in rd else np.full_like(mu_lin_oof, 100.0)
        abs_wsb = np.nan_to_num(rd['oof_abs_wsb'][vm].astype(float), nan=0.0) if 'oof_abs_wsb' in rd else np.zeros_like(mu_lin_oof)
        sig_raw = np.maximum(d*sig_q + (1-d)*sig_cat, 5.0)
        sigma_oof = np.maximum(sig_raw*s, a + b*np.clip(abs_wsb, 0, 40))
        sigma_oof = np.maximum(sigma_oof, c*prmse_oof)
        sigma_oof = np.maximum(sigma_oof, 70.0 + e*np.clip(np.abs(wsl_oof), 0, 12))
        sigma_oof = np.maximum(sigma_oof, 70.0)
        base_score = laplace_log_likelihood(y_oof, mu_lin_oof + 0.30*res_cat_oof + 0.70*res_lgbm_oof, sigma_oof, sigma_floor=70.0)
        adj_score  = laplace_log_likelihood(y_oof, mu_oof_new, sigma_oof, sigma_floor=70.0)
        print(f"OOF proxy: base={base_score:.5f} -> mu-adjusted={adj_score:.5f} (delta={adj_score-base_score:+.5f})")
    else:
        print("OOF proxy skipped (res3_5/res3_10 not found).")
except Exception as _e:
    print("OOF proxy failed:", _e)

print("Gate: seek OOF proxy delta >= +0.12 vs base (target ~ -6.86 or better for bronze).")

# 11) Write submission with new mu (sigma unchanged) for Preset-2
try:      
    out = ss.copy()
    out['FVC'] = mu_final.astype(float)
    out['Confidence'] = sigma.astype(float)  # sigma computed earlier; untouched here
    out.to_csv('submission_preset2.csv', index=False)
    print('Saved submission_preset2.csv (Preset-2, sigma unchanged).')
except Exception as e:
    print('Failed to write submission_preset2.csv:', e)
# ===================== END MU-ONLY PATCH =======================

Policy diagnostics -> N: 1908 vcount>=1: 1556 |wsl|<=30: 892 pct [55,92]: 1060 |anchor_resid|<=150: 1908 var_prior<=260: 1908 |rs2|<=45: 1908 all: 300
MU policy counts -> aggressive: 300, conservative: 1608 (override=None)
mu_final - mu_lin_sub five-number summary: [-56.2 -46.4 -35.2  37.8  67.7]
Aggressive delta mu five-number: [-61.2 -52.9  -1.6  40.6  46.5]
Conservative delta mu five-number: [-53.9 -46.4 -35.2  37.8  67.7]
OOF proxy: base=-6.98952 -> mu-adjusted=-6.86214 (delta=+0.12738)
Gate: seek OOF proxy delta >= +0.12 vs base (target ~ -6.86 or better for bronze).
Saved submission_preset2.csv (Preset-2, sigma unchanged).


In [215]:
# Copy Preset-2 variant to submission.csv and show quick stats
import pandas as pd, numpy as np, shutil, os
src, dst = 'submission_preset2.csv', 'submission.csv'
assert os.path.exists(src), 'submission_preset2.csv not found'
shutil.copyfile(src, dst)
sub = pd.read_csv(dst)
print('Rows:', len(sub), 'NaNs:', sub.isna().sum().to_dict())
print('Confidence pct [10,25,50,75,90]:', np.percentile(sub['Confidence'].values.astype(float), [10,25,50,75,90]).round(1))
print(sub.head(3))

Rows: 1908 NaNs: {'Patient_Week': 0, 'FVC': 0, 'Confidence': 0}
Confidence pct [10,25,50,75,90]: [230.  248.  251.6 305.2 545.4]
                   Patient_Week     FVC  Confidence
0  ID00126637202218610655908_-3  2607.0  468.771497
1  ID00126637202218610655908_-2  2607.0  468.771508
2  ID00126637202218610655908_-1  2607.0  468.771457


In [195]:
# Hedge run: tighten MU clamp to K=50 and regenerate Preset-2 submission
import numpy as np, pandas as pd

K = 50.0
mu_pp = np.clip(mu_pp, mu_lin_sub - K, mu_lin_sub + K)
mu_base = mu_pp.copy()
mu_final = apply_caps_and_debias(mu_base)

try:
    out = ss.copy()
    out['FVC'] = mu_final.astype(float)
    out['Confidence'] = sigma.astype(float)  # sigma unchanged
    out.to_csv('submission_preset2.csv', index=False)
    print('HEDGE (K=50) -> Saved submission_preset2.csv')
    # quick diag
    delta = mu_final - mu_lin_sub
    def p5(x):
        return np.percentile(x, [5,25,50,75,95]).round(1)
    print('mu_final - mu_lin_sub five-number (K=50):', p5(delta))
except Exception as e:
    print('HEDGE (K=50) failed:', e)

HEDGE (K=50) -> Saved submission_preset2.csv
mu_final - mu_lin_sub five-number (K=50): [-46.5 -38.  -34.3  37.8  67.7]


In [197]:
# Anchor blend hedge: mu_base = 0.8*mu_pp + 0.2*mu_lin_sub (keep K=60 and sigma unchanged); regenerate Preset-2
import numpy as np, pandas as pd

mu_base = 0.8 * mu_pp + 0.2 * mu_lin_sub
mu_final = apply_caps_and_debias(mu_base)

try:
    out = ss.copy()
    out['FVC'] = mu_final.astype(float)
    out['Confidence'] = sigma.astype(float)  # sigma unchanged
    out.to_csv('submission_preset2.csv', index=False)
    print('ANCHOR BLEND (0.8/0.2) -> Saved submission_preset2.csv')
    delta = mu_final - mu_lin_sub
    def p5(x):
        return np.percentile(x, [5,25,50,75,95]).round(1)
    print('mu_final - mu_lin_sub five-number (anchor 0.8/0.2):', p5(delta))
except Exception as e:
    print('ANCHOR BLEND (0.8/0.2) failed:', e)

ANCHOR BLEND (0.8/0.2) -> Saved submission_preset2.csv
mu_final - mu_lin_sub five-number (anchor 0.8/0.2): [-37.2 -31.2 -29.6  33.6  67.7]


In [199]:
# Anchor blend hedge 2: mu_base = 0.7*mu_pp + 0.3*mu_lin_sub; regenerate Preset-2
import numpy as np, pandas as pd

mu_base = 0.7 * mu_pp + 0.3 * mu_lin_sub
mu_final = apply_caps_and_debias(mu_base)

try:
    out = ss.copy()
    out['FVC'] = mu_final.astype(float)
    out['Confidence'] = sigma.astype(float)  # sigma unchanged
    out.to_csv('submission_preset2.csv', index=False)
    print('ANCHOR BLEND (0.7/0.3) -> Saved submission_preset2.csv')
    delta = mu_final - mu_lin_sub
    def p5(x):
        return np.percentile(x, [5,25,50,75,95]).round(1)
    print('mu_final - mu_lin_sub five-number (anchor 0.7/0.3):', p5(delta))
except Exception as e:
    print('ANCHOR BLEND (0.7/0.3) failed:', e)

ANCHOR BLEND (0.7/0.3) -> Saved submission_preset2.csv
mu_final - mu_lin_sub five-number (anchor 0.7/0.3): [-34.3 -27.5 -25.4  29.4  67.7]


In [201]:
# Recompute mu_pp with K=60 and apply anchor blend 0.8/0.2; regenerate Preset-2
import numpy as np, pandas as pd

# Rebuild mu_pp from existing mu1/mu2 (computed in Cell 29)
mu_pp = 0.5 * mu1 + 0.5 * mu2
K = 60.0
mu_pp = np.clip(mu_pp.astype(float), mu_lin_sub - K, mu_lin_sub + K)

# Anchor blend per expert: 0.8/0.2
mu_base = 0.8 * mu_pp + 0.2 * mu_lin_sub
mu_final = apply_caps_and_debias(mu_base)

try:
    out = ss.copy()
    out['FVC'] = mu_final.astype(float)
    out['Confidence'] = sigma.astype(float)  # sigma unchanged
    out.to_csv('submission_preset2.csv', index=False)
    print('RECOMP (K=60) + ANCHOR (0.8/0.2) -> Saved submission_preset2.csv')
    delta = mu_final - mu_lin_sub
    def p5(x):
        return np.percentile(x, [5,25,50,75,95]).round(1)
    print('mu_final - mu_lin_sub five-number (K=60, 0.8/0.2):', p5(delta))
except Exception as e:
    print('RECOMP (K=60) + ANCHOR (0.8/0.2) failed:', e)

RECOMP (K=60) + ANCHOR (0.8/0.2) -> Saved submission_preset2.csv
mu_final - mu_lin_sub five-number (K=60, 0.8/0.2): [-44.6 -36.3 -34.3  37.8  67.7]


In [203]:
# Anchor blend per expert: mu_base = 0.6*mu_pp + 0.4*mu_lin_sub (keep K=60); regenerate Preset-2
import numpy as np, pandas as pd

mu_base = 0.6 * mu_pp + 0.4 * mu_lin_sub
mu_final = apply_caps_and_debias(mu_base)

try:
    out = ss.copy()
    out['FVC'] = mu_final.astype(float)
    out['Confidence'] = sigma.astype(float)  # sigma unchanged
    out.to_csv('submission_preset2.csv', index=False)
    print('ANCHOR BLEND (0.6/0.4) -> Saved submission_preset2.csv')
    delta = mu_final - mu_lin_sub
    def p5(x):
        return np.percentile(x, [5,25,50,75,95]).round(1)
    print('mu_final - mu_lin_sub five-number (anchor 0.6/0.4):', p5(delta))
except Exception as e:
    print('ANCHOR BLEND (0.6/0.4) failed:', e)

ANCHOR BLEND (0.6/0.4) -> Saved submission_preset2.csv
mu_final - mu_lin_sub five-number (anchor 0.6/0.4): [-34.3 -28.3 -26.2  30.2  67.7]


In [206]:
# Final fallback hedge: use mu_lin_sub baseline (ignore MU1+MU2), keep sigma unchanged; regenerate Preset-2
import numpy as np, pandas as pd

mu_base = mu_lin_sub.copy().astype(float)
mu_final = apply_caps_and_debias(mu_base)

try:
    out = ss.copy()
    out['FVC'] = mu_final.astype(float)
    out['Confidence'] = sigma.astype(float)  # sigma unchanged
    out.to_csv('submission_preset2.csv', index=False)
    print('FALLBACK (mu_lin_sub) -> Saved submission_preset2.csv')
    delta = mu_final - mu_lin_sub
    def p5(x):
        return np.percentile(x, [5,25,50,75,95]).round(1)
    print('mu_final - mu_lin_sub five-number (fallback):', p5(delta))
except Exception as e:
    print('FALLBACK (mu_lin_sub) failed:', e)

FALLBACK (mu_lin_sub) -> Saved submission_preset2.csv
mu_final - mu_lin_sub five-number (fallback): [-11.2   0.    4.    4.   67.7]


In [208]:
# Anchor-stick policy fallback: redefine caps/debias and rebuild submission_preset2.csv
import numpy as np, pandas as pd

def apply_caps_and_debias(mu_in):
    # Anchor-stick near last-known; mild cap/window elsewhere
    mu = mu_in.copy().astype(float)

    # 1) Stick to last_known when very close to anchor
    stick_mask = (np.abs(wsl_anchor) <= 2.0)
    if np.any(stick_mask):
        mu[stick_mask] = last_known[stick_mask]

    # 2) Mild upward cap vs last_known
    cap_up = last_known + 70.0
    mu = np.minimum(mu, cap_up)

    # 3) Baseline window ±180 around baseline_FVC
    mu = np.clip(mu, base_fvc - 180.0, base_fvc + 180.0)

    # 4) Lower guard vs last_known
    mu = np.maximum(mu, last_known - 380.0)

    return mu

# Use pure mu_lin_sub baseline as input
mu_base = mu_lin_sub.copy().astype(float)
mu_final = apply_caps_and_debias(mu_base)

try:
    out = ss.copy()
    out['FVC'] = mu_final.astype(float)
    out['Confidence'] = sigma.astype(float)  # sigma unchanged
    out.to_csv('submission_preset2.csv', index=False)
    print('ANCHOR-STICK fallback -> Saved submission_preset2.csv')
    delta = mu_final - mu_lin_sub
    def p5(x):
        return np.percentile(x, [5,25,50,75,95]).round(1)
    print('mu_final - mu_lin_sub five-number (anchor-stick):', p5(delta))
except Exception as e:
    print('ANCHOR-STICK fallback failed:', e)

ANCHOR-STICK fallback -> Saved submission_preset2.csv
mu_final - mu_lin_sub five-number (anchor-stick): [-52.   0.   0.   0. 450.]


In [210]:
# Identity MU override: use pure mu_lin_sub, keep sigma unchanged; regenerate Preset-2
import numpy as np, pandas as pd

# IDENTITY MU: no caps/debias, pure linear baseline
mu_final = np.asarray(mu_lin_sub, dtype=float).copy()

out = ss.copy()
out['FVC'] = mu_final.astype(float)
out['Confidence'] = sigma.astype(float)  # sigma stays as-is
out.to_csv('submission_preset2.csv', index=False)
print('Saved submission_preset2.csv (Identity MU, sigma unchanged)')
print('delta(mu_final - mu_lin_sub) five-number:', np.percentile(mu_final - mu_lin_sub, [5,25,50,75,95]).round(1))

Saved submission_preset2.csv (Identity MU, sigma unchanged)
delta(mu_final - mu_lin_sub) five-number: [0. 0. 0. 0. 0.]


In [212]:
# Identity + tiny long-gap shrink (gamma=0.04 when |wsl_anchor|>=24), else identity; regenerate Preset-2
import numpy as np, pandas as pd

last_known = np.nan_to_num(
    X_sub_feats.get('last_known_FVC', pd.Series(np.nan, index=X_sub_feats.index)).values.astype(float),
    nan=mu_lin_sub
)
wsl_anchor = np.nan_to_num(
    X_sub_feats.get('weeks_since_anchor', pd.Series(0.0, index=X_sub_feats.index)).values.astype(float),
    nan=0.0
)
gamma = np.where(np.abs(wsl_anchor) >= 24.0, 0.04, 0.0).astype(float)
mu_final = (1.0 - gamma) * mu_lin_sub + gamma * last_known

out = ss.copy()
out['FVC'] = mu_final.astype(float)
out['Confidence'] = sigma.astype(float)  # keep sigma unchanged
out.to_csv('submission_preset2.csv', index=False)
print('Saved submission_preset2.csv (Identity + tiny long-gap shrink). Long-gap rows:',
      int((np.abs(wsl_anchor) >= 24.0).sum()))

Saved submission_preset2.csv (Identity + tiny long-gap shrink). Long-gap rows: 1142


In [214]:
# Identity + near-anchor stick only; regenerate Preset-2
import numpy as np, pandas as pd

# If |weeks_since_anchor| <= 2 -> use last_known_FVC; else use mu_lin_sub (sigma unchanged)
last_known = np.nan_to_num(
    X_sub_feats['last_known_FVC'].values.astype(float),
    nan=mu_lin_sub
)
wsl_anchor = np.nan_to_num(
    X_sub_feats['weeks_since_anchor'].values.astype(float),
    nan=0.0
)
mask = (np.abs(wsl_anchor) <= 2.0)
mu_final = np.asarray(mu_lin_sub, dtype=float).copy()
if np.any(mask):
    mu_final[mask] = last_known[mask]

out = ss.copy()
out['FVC'] = mu_final.astype(float)
out['Confidence'] = sigma.astype(float)  # sigma untouched
out.to_csv('submission_preset2.csv', index=False)
print('Saved submission_preset2.csv (Identity + near-anchor stick). Stick rows:', int(mask.sum()))

Saved submission_preset2.csv (Identity + near-anchor stick). Stick rows: 388
